# Load data from Azure Data Lake Storage to SQL Data Warehouse 

## Create a Database Master Key
Only necessary if one does not already exist.
Required to encrypt the credential secret in the next step.

In [37]:
CREATE MASTER KEY;

## Create a database scoped credential
### For Service Principal Authentication

In [38]:
CREATE DATABASE SCOPED CREDENTIAL AADCredential
WITH
    IDENTITY = '<client-id>@https://login.microsoftonline.com/<tenant-id>/oauth2/token',
    SECRET = '<client-secret>'
;

### For Storage Key Authentication

In [39]:
CREATE DATABASE SCOPED CREDENTIAL ADLSStorageKey
WITH
    IDENTITY = 'user',
    SECRET = '<storage-account-key>'
;

## Create the external data sources

In [40]:
CREATE EXTERNAL DATA SOURCE AzureDataLakeStorageGen1
WITH (
    TYPE = HADOOP,
    LOCATION = 'adl://<adls-gen1-name>.azuredatalakestore.net',
    CREDENTIAL = AADCredential
);

In [41]:
CREATE EXTERNAL DATA SOURCE AzureDataLakeStorageGen2
WITH (
    TYPE = HADOOP,
    LOCATION='abfss://<container-name>@<adls-gen2-name>.dfs.core.windows.net', 
    CREDENTIAL = ADLSStorageKey
);

## Configure data format

In [42]:
CREATE EXTERNAL FILE FORMAT TextFileFormat 
WITH 
(   FORMAT_TYPE = DELIMITEDTEXT
,	FORMAT_OPTIONS	(   FIELD_TERMINATOR = '|'
					,	STRING_DELIMITER = ''
					,	DATE_FORMAT		 = 'yyyy-MM-dd HH:mm:ss.fff'
					,	USE_TYPE_DEFAULT = FALSE 
					)
);

## Create the External Tables

In [43]:
CREATE SCHEMA [adls1];

In [44]:
--DimAccount
CREATE EXTERNAL TABLE [adls1].DimAccount 
(
	[AccountKey] [int] NOT NULL,
	[ParentAccountKey] [int] NULL,
	[AccountLabel] [nvarchar](100) NULL,
	[AccountName] [nvarchar](50) NULL,
	[AccountDescription] [nvarchar](50) NULL,
	[AccountType] [nvarchar](50) NULL,
	[Operator] [nvarchar](50) NULL,
	[CustomMembers] [nvarchar](300) NULL,
	[ValueType] [nvarchar](50) NULL,
	[CustomMemberOptions] [nvarchar](200) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH 
(
    LOCATION='/contosoretaildw/DimAccount/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO 
--DimChannel
CREATE EXTERNAL TABLE [adls1].DimChannel 
(
	[ChannelKey] [int] NOT NULL,
	[ChannelLabel] [nvarchar](100) NOT NULL,
	[ChannelName] [nvarchar](20) NULL,
	[ChannelDescription] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimChannel/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimCurrency
CREATE EXTERNAL TABLE [adls1].DimCurrency 
(
	[CurrencyKey] [int] NOT NULL,
	[CurrencyLabel] [nvarchar](10) NOT NULL,
	[CurrencyName] [nvarchar](20) NOT NULL,
	[CurrencyDescription] [nvarchar](50) NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimCurrency/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimCustomer
CREATE EXTERNAL TABLE [adls1].DimCustomer 
(
	[CustomerKey] [int]  NOT NULL,
	[GeographyKey] [int] NOT NULL,
	[CustomerLabel] [nvarchar](100) NOT NULL,
	[Title] [nvarchar](8) NULL,
	[FirstName] [nvarchar](50) NULL,
	[MiddleName] [nvarchar](50) NULL,
	[LastName] [nvarchar](50) NULL,
	[NameStyle] [bit] NULL,
	[BirthDate] [datetime] NULL,
	[MaritalStatus] [nchar](1) NULL,
	[Suffix] [nvarchar](10) NULL,
	[Gender] [nvarchar](1) NULL,
	[EmailAddress] [nvarchar](50) NULL,
	[YearlyIncome] [money] NULL,
	[TotalChildren] [tinyint] NULL,
	[NumberChildrenAtHome] [tinyint] NULL,
	[Education] [nvarchar](40) NULL,
	[Occupation] [nvarchar](100) NULL,
	[HouseOwnerFlag] [nchar](1) NULL,
	[NumberCarsOwned] [tinyint] NULL,
	[AddressLine1] [nvarchar](120) NULL,
	[AddressLine2] [nvarchar](120) NULL,
	[Phone] [nvarchar](20) NULL,
	[DateFirstPurchase] [datetime] NULL,
	[CustomerType] [nvarchar](15) NULL,
	[CompanyName] [nvarchar](100) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimCustomer/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimDate
CREATE EXTERNAL TABLE [adls1].DimDate
(
	[Datekey] [datetime] NOT NULL,
	[FullDateLabel] [nvarchar](20) NOT NULL,
	[DateDescription] [nvarchar](20) NOT NULL,
	[CalendarYear] [int] NOT NULL,
	[CalendarYearLabel] [nvarchar](20) NOT NULL,
	[CalendarHalfYear] [int] NOT NULL,
	[CalendarHalfYearLabel] [nvarchar](20) NOT NULL,
	[CalendarQuarter] [int] NOT NULL,
	[CalendarQuarterLabel] [nvarchar](20) NULL,
	[CalendarMonth] [int] NOT NULL,
	[CalendarMonthLabel] [nvarchar](20) NOT NULL,
	[CalendarWeek] [int] NOT NULL,
	[CalendarWeekLabel] [nvarchar](20) NOT NULL,
	[CalendarDayOfWeek] [int] NOT NULL,
	[CalendarDayOfWeekLabel] [nvarchar](10) NOT NULL,
	[FiscalYear] [int] NOT NULL,
	[FiscalYearLabel] [nvarchar](20) NOT NULL,
	[FiscalHalfYear] [int] NOT NULL,
	[FiscalHalfYearLabel] [nvarchar](20) NOT NULL,
	[FiscalQuarter] [int] NOT NULL,
	[FiscalQuarterLabel] [nvarchar](20) NOT NULL,
	[FiscalMonth] [int] NOT NULL,
	[FiscalMonthLabel] [nvarchar](20) NOT NULL,
	[IsWorkDay] [nvarchar](20) NOT NULL,
	[IsHoliday] [int] NOT NULL,
	[HolidayName] [nvarchar](20) NOT NULL,
	[EuropeSeason] [nvarchar](50) NULL,
	[NorthAmericaSeason] [nvarchar](50) NULL,
	[AsiaSeason] [nvarchar](50) NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimDate/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimEmployee
CREATE EXTERNAL TABLE [adls1].DimEmployee 
(
	[EmployeeKey] [int]  NOT NULL,
	[ParentEmployeeKey] [int] NULL,
	[FirstName] [nvarchar](50) NOT NULL,
	[LastName] [nvarchar](50) NOT NULL,
	[MiddleName] [nvarchar](50) NULL,
	[Title] [nvarchar](50) NULL,
	[HireDate] [datetime] NULL,
	[BirthDate] [datetime] NULL,
	[EmailAddress] [nvarchar](50) NULL,
	[Phone] [nvarchar](25) NULL,
	[MaritalStatus] [nchar](1) NULL,
	[EmergencyContactName] [nvarchar](50) NULL,
	[EmergencyContactPhone] [nvarchar](25) NULL,
	[SalariedFlag] [bit] NULL,
	[Gender] [nchar](1) NULL,
	[PayFrequency] [tinyint] NULL,
	[BaseRate] [money] NULL,
	[VacationHours] [smallint] NULL,
	[CurrentFlag] [bit] NOT NULL,
	[SalesPersonFlag] [bit] NOT NULL,
	[DepartmentName] [nvarchar](50) NULL,
	[StartDate] [datetime] NULL,
	[EndDate] [datetime] NULL,
	[Status] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimEmployee/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimEntity
CREATE EXTERNAL TABLE [adls1].DimEntity 
(
	[EntityKey] [int] NOT NULL,
	[EntityLabel] [nvarchar](100) NULL,
	[ParentEntityKey] [int] NULL,
	[ParentEntityLabel] [nvarchar](100) NULL,
	[EntityName] [nvarchar](50) NULL,
	[EntityDescription] [nvarchar](100) NULL,
	[EntityType] [nvarchar](100) NULL,
	[StartDate] [datetime] NULL,
	[EndDate] [datetime] NULL,
	[Status] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimEntity/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimGeography
CREATE EXTERNAL TABLE [adls1].DimGeography 
(
	[GeographyKey] [int] NOT NULL,
	[GeographyType] [nvarchar](50) NOT NULL,
	[ContinentName] [nvarchar](50) NOT NULL,
	[CityName] [nvarchar](100) NULL,
	[StateProvinceName] [nvarchar](100) NULL,
	[RegionCountryName] [nvarchar](100) NULL,
--	[Geometry] [geometry] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimGeography/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimMachine
CREATE EXTERNAL TABLE [adls1].DimMachine 
(
	[MachineKey] [int] NOT NULL,
	[MachineLabel] [nvarchar](100) NULL,
	[StoreKey] [int] NOT NULL,
	[MachineType] [nvarchar](50) NOT NULL,
	[MachineName] [nvarchar](100) NOT NULL,
	[MachineDescription] [nvarchar](200) NOT NULL,
	[VendorName] [nvarchar](50) NOT NULL,
	[MachineOS] [nvarchar](50) NOT NULL,
	[MachineSource] [nvarchar](100) NOT NULL,
	[MachineHardware] [nvarchar](100) NULL,
	[MachineSoftware] [nvarchar](100) NOT NULL,
	[Status] [nvarchar](50) NOT NULL,
	[ServiceStartDate] [datetime] NOT NULL,
	[DecommissionDate] [datetime] NULL,
	[LastModifiedDate] [datetime] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimMachine/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimOutage
CREATE EXTERNAL TABLE [adls1].DimOutage (
	[OutageKey] [int]  NOT NULL,
	[OutageLabel] [nvarchar](100) NOT NULL,
	[OutageName] [nvarchar](50) NOT NULL,
	[OutageDescription] [nvarchar](200) NOT NULL,
	[OutageType] [nvarchar](50) NOT NULL,
	[OutageTypeDescription] [nvarchar](200) NOT NULL,
	[OutageSubType] [nvarchar](50) NOT NULL,
	[OutageSubTypeDescription] [nvarchar](200) NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimOutage/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimProduct
CREATE EXTERNAL TABLE [adls1].DimProduct (
	[ProductKey] [int] NOT NULL,
	[ProductLabel] [nvarchar](255) NULL,
	[ProductName] [nvarchar](500) NULL,
	[ProductDescription] [nvarchar](400) NULL,
	[ProductSubcategoryKey] [int] NULL,
	[Manufacturer] [nvarchar](50) NULL,
	[BrandName] [nvarchar](50) NULL,
	[ClassID] [nvarchar](10) NULL,
	[ClassName] [nvarchar](20) NULL,
	[StyleID] [nvarchar](10) NULL,
	[StyleName] [nvarchar](20) NULL,
	[ColorID] [nvarchar](10) NULL,
	[ColorName] [nvarchar](20) NOT NULL,
	[Size] [nvarchar](50) NULL,
	[SizeRange] [nvarchar](50) NULL,
	[SizeUnitMeasureID] [nvarchar](20) NULL,
	[Weight] [float] NULL,
	[WeightUnitMeasureID] [nvarchar](20) NULL,
	[UnitOfMeasureID] [nvarchar](10) NULL,
	[UnitOfMeasureName] [nvarchar](40) NULL,
	[StockTypeID] [nvarchar](10) NULL,
	[StockTypeName] [nvarchar](40) NULL,
	[UnitCost] [money] NULL,
	[UnitPrice] [money] NULL,
	[AvailableForSaleDate] [datetime] NULL,
	[StopSaleDate] [datetime] NULL,
	[Status] [nvarchar](7) NULL,
	[ImageURL] [nvarchar](150) NULL,
	[ProductURL] [nvarchar](150) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimProduct/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimProductCategory
CREATE EXTERNAL TABLE [adls1].DimProductCategory (
	[ProductCategoryKey] [int]  NOT NULL,
	[ProductCategoryLabel] [nvarchar](100) NULL,
	[ProductCategoryName] [nvarchar](30) NOT NULL,
	[ProductCategoryDescription] [nvarchar](50) NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimProductCategory/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimProductSubcategory
CREATE EXTERNAL TABLE [adls1].DimProductSubcategory (
	[ProductSubcategoryKey] [int]  NOT NULL,
	[ProductSubcategoryLabel] [nvarchar](100) NULL,
	[ProductSubcategoryName] [nvarchar](50) NOT NULL,
	[ProductSubcategoryDescription] [nvarchar](100) NULL,
	[ProductCategoryKey] [int] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimProductSubcategory/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimPromotion
CREATE EXTERNAL TABLE [adls1].DimPromotion (
	[PromotionKey] [int]  NOT NULL,
	[PromotionLabel] [nvarchar](100) NULL,
	[PromotionName] [nvarchar](100) NULL,
	[PromotionDescription] [nvarchar](255) NULL,
	[DiscountPercent] [float] NULL,
	[PromotionType] [nvarchar](50) NULL,
	[PromotionCategory] [nvarchar](50) NULL,
	[StartDate] [datetime] NOT NULL,
	[EndDate] [datetime] NULL,
	[MinQuantity] [int] NULL,
	[MaxQuantity] [int] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimPromotion/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimSalesTerritory
CREATE EXTERNAL TABLE [adls1].DimSalesTerritory (
	[SalesTerritoryKey] [int]  NOT NULL,
	[GeographyKey] [int] NOT NULL,
	[SalesTerritoryLabel] [nvarchar](100) NULL,
	[SalesTerritoryName] [nvarchar](50) NOT NULL,
	[SalesTerritoryRegion] [nvarchar](50) NOT NULL,
	[SalesTerritoryCountry] [nvarchar](50) NOT NULL,
	[SalesTerritoryGroup] [nvarchar](50) NULL,
	[SalesTerritoryLevel] [nvarchar](10) NULL,
	[SalesTerritoryManager] [int] NULL,
	[StartDate] [datetime] NULL,
	[EndDate] [datetime] NULL,
	[Status] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimSalesTerritory/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimScenario
CREATE EXTERNAL TABLE [adls1].DimScenario (
	[ScenarioKey] [int] NOT NULL,
	[ScenarioLabel] [nvarchar](100) NOT NULL,
	[ScenarioName] [nvarchar](20) NULL,
	[ScenarioDescription] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimScenario/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimStore
CREATE EXTERNAL TABLE [adls1].DimStore 
(
	[StoreKey] [int] NOT NULL,
	[GeographyKey] [int] NOT NULL,
	[StoreManager] [int] NULL,
	[StoreType] [nvarchar](15) NULL,
	[StoreName] [nvarchar](100) NOT NULL,
	[StoreDescription] [nvarchar](300) NOT NULL,
	[Status] [nvarchar](20) NOT NULL,
	[OpenDate] [datetime] NOT NULL,
	[CloseDate] [datetime] NULL,
	[EntityKey] [int] NULL,
	[ZipCode] [nvarchar](20) NULL,
	[ZipCodeExtension] [nvarchar](10) NULL,
	[StorePhone] [nvarchar](15) NULL,
	[StoreFax] [nvarchar](14) NULL,
	[AddressLine1] [nvarchar](100) NULL,
	[AddressLine2] [nvarchar](100) NULL,
	[CloseReason] [nvarchar](20) NULL,
	[EmployeeCount] [int] NULL,
	[SellingAreaSize] [float] NULL,
	[LastRemodelDate] [datetime] NULL,
	[GeoLocation]	NVARCHAR(50)  NULL,
	[Geometry]		NVARCHAR(50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/DimStore/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactExchangeRate
CREATE EXTERNAL TABLE [adls1].FactExchangeRate 
(
	[ExchangeRateKey] [int]  NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[AverageRate] [float] NOT NULL,
	[EndOfDayRate] [float] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/FactExchangeRate/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactInventory
CREATE EXTERNAL TABLE [adls1].FactInventory (
	[InventoryKey] [int]  NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[ProductKey] [int] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[OnHandQuantity] [int] NOT NULL,
	[OnOrderQuantity] [int] NOT NULL,
	[SafetyStockQuantity] [int] NULL,
	[UnitCost] [money] NOT NULL,
	[DaysInStock] [int] NULL,
	[MinDayInStock] [int] NULL,
	[MaxDayInStock] [int] NULL,
	[Aging] [int] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/FactInventory/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactITMachine
CREATE EXTERNAL TABLE [adls1].FactITMachine (
	[ITMachinekey] [int] NOT NULL,
	[MachineKey] [int] NOT NULL,
	[Datekey] [datetime] NOT NULL,
	[CostAmount] [money] NULL,
	[CostType] [nvarchar](200) NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/FactITMachine/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactITSLA
CREATE EXTERNAL TABLE [adls1].FactITSLA 
(
	[ITSLAkey] [int]  NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[MachineKey] [int] NOT NULL,
	[OutageKey] [int] NOT NULL,
	[OutageStartTime] [datetime] NOT NULL,
	[OutageEndTime] [datetime] NOT NULL,
	[DownTime] [int] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/FactITSLA/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactOnlineSales
CREATE EXTERNAL TABLE [adls1].FactOnlineSales 
(
	[OnlineSalesKey] [int]  NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[ProductKey] [int] NOT NULL,
	[PromotionKey] [int] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[CustomerKey] [int] NOT NULL,
	[SalesOrderNumber] [nvarchar](20) NOT NULL,
	[SalesOrderLineNumber] [int] NULL,
	[SalesQuantity] [int] NOT NULL,
	[SalesAmount] [money] NOT NULL,
	[ReturnQuantity] [int] NOT NULL,
	[ReturnAmount] [money] NULL,
	[DiscountQuantity] [int] NULL,
	[DiscountAmount] [money] NULL,
	[TotalCost] [money] NOT NULL,
	[UnitCost] [money] NULL,
	[UnitPrice] [money] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/FactOnlineSales/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactSales
CREATE EXTERNAL TABLE [adls1].FactSales 
(
	[SalesKey] [int]  NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[channelKey] [int] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[ProductKey] [int] NOT NULL,
	[PromotionKey] [int] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[UnitCost] [money] NOT NULL,
	[UnitPrice] [money] NOT NULL,
	[SalesQuantity] [int] NOT NULL,
	[ReturnQuantity] [int] NOT NULL,
	[ReturnAmount] [money] NULL,
	[DiscountQuantity] [int] NULL,
	[DiscountAmount] [money] NULL,
	[TotalCost] [money] NOT NULL,
	[SalesAmount] [money] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/FactSales/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactSalesQuota
CREATE EXTERNAL TABLE [adls1].FactSalesQuota (
	[SalesQuotaKey] [int]  NOT NULL,
	[ChannelKey] [int] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[ProductKey] [int] NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[ScenarioKey] [int] NOT NULL,
	[SalesQuantityQuota] [money] NOT NULL,
	[SalesAmountQuota] [money] NOT NULL,
	[GrossMarginQuota] [money] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/FactSalesQuota/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactStrategyPlan
CREATE EXTERNAL TABLE [adls1].FactStrategyPlan 
(
	[StrategyPlanKey] [int]  NOT NULL,
	[Datekey] [datetime] NOT NULL,
	[EntityKey] [int] NOT NULL,
	[ScenarioKey] [int] NOT NULL,
	[AccountKey] [int] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[ProductCategoryKey] [int] NULL,
	[Amount] [money] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/contosoretaildw/FactStrategyPlan/' 
,   DATA_SOURCE = AzureDataLakeStorageGen1
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;

In [45]:
CREATE SCHEMA [adls2];

In [46]:
--DimAccount
CREATE EXTERNAL TABLE [adls2].DimAccount 
(
	[AccountKey] [int] NOT NULL,
	[ParentAccountKey] [int] NULL,
	[AccountLabel] [nvarchar](100) NULL,
	[AccountName] [nvarchar](50) NULL,
	[AccountDescription] [nvarchar](50) NULL,
	[AccountType] [nvarchar](50) NULL,
	[Operator] [nvarchar](50) NULL,
	[CustomMembers] [nvarchar](300) NULL,
	[ValueType] [nvarchar](50) NULL,
	[CustomMemberOptions] [nvarchar](200) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH 
(
    LOCATION='/DimAccount/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO 
--DimChannel
CREATE EXTERNAL TABLE [adls2].DimChannel 
(
	[ChannelKey] [int] NOT NULL,
	[ChannelLabel] [nvarchar](100) NOT NULL,
	[ChannelName] [nvarchar](20) NULL,
	[ChannelDescription] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimChannel/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimCurrency
CREATE EXTERNAL TABLE [adls2].DimCurrency 
(
	[CurrencyKey] [int] NOT NULL,
	[CurrencyLabel] [nvarchar](10) NOT NULL,
	[CurrencyName] [nvarchar](20) NOT NULL,
	[CurrencyDescription] [nvarchar](50) NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimCurrency/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimCustomer
CREATE EXTERNAL TABLE [adls2].DimCustomer 
(
	[CustomerKey] [int]  NOT NULL,
	[GeographyKey] [int] NOT NULL,
	[CustomerLabel] [nvarchar](100) NOT NULL,
	[Title] [nvarchar](8) NULL,
	[FirstName] [nvarchar](50) NULL,
	[MiddleName] [nvarchar](50) NULL,
	[LastName] [nvarchar](50) NULL,
	[NameStyle] [bit] NULL,
	[BirthDate] [datetime] NULL,
	[MaritalStatus] [nchar](1) NULL,
	[Suffix] [nvarchar](10) NULL,
	[Gender] [nvarchar](1) NULL,
	[EmailAddress] [nvarchar](50) NULL,
	[YearlyIncome] [money] NULL,
	[TotalChildren] [tinyint] NULL,
	[NumberChildrenAtHome] [tinyint] NULL,
	[Education] [nvarchar](40) NULL,
	[Occupation] [nvarchar](100) NULL,
	[HouseOwnerFlag] [nchar](1) NULL,
	[NumberCarsOwned] [tinyint] NULL,
	[AddressLine1] [nvarchar](120) NULL,
	[AddressLine2] [nvarchar](120) NULL,
	[Phone] [nvarchar](20) NULL,
	[DateFirstPurchase] [datetime] NULL,
	[CustomerType] [nvarchar](15) NULL,
	[CompanyName] [nvarchar](100) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimCustomer/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimDate
CREATE EXTERNAL TABLE [adls2].DimDate
(
	[Datekey] [datetime] NOT NULL,
	[FullDateLabel] [nvarchar](20) NOT NULL,
	[DateDescription] [nvarchar](20) NOT NULL,
	[CalendarYear] [int] NOT NULL,
	[CalendarYearLabel] [nvarchar](20) NOT NULL,
	[CalendarHalfYear] [int] NOT NULL,
	[CalendarHalfYearLabel] [nvarchar](20) NOT NULL,
	[CalendarQuarter] [int] NOT NULL,
	[CalendarQuarterLabel] [nvarchar](20) NULL,
	[CalendarMonth] [int] NOT NULL,
	[CalendarMonthLabel] [nvarchar](20) NOT NULL,
	[CalendarWeek] [int] NOT NULL,
	[CalendarWeekLabel] [nvarchar](20) NOT NULL,
	[CalendarDayOfWeek] [int] NOT NULL,
	[CalendarDayOfWeekLabel] [nvarchar](10) NOT NULL,
	[FiscalYear] [int] NOT NULL,
	[FiscalYearLabel] [nvarchar](20) NOT NULL,
	[FiscalHalfYear] [int] NOT NULL,
	[FiscalHalfYearLabel] [nvarchar](20) NOT NULL,
	[FiscalQuarter] [int] NOT NULL,
	[FiscalQuarterLabel] [nvarchar](20) NOT NULL,
	[FiscalMonth] [int] NOT NULL,
	[FiscalMonthLabel] [nvarchar](20) NOT NULL,
	[IsWorkDay] [nvarchar](20) NOT NULL,
	[IsHoliday] [int] NOT NULL,
	[HolidayName] [nvarchar](20) NOT NULL,
	[EuropeSeason] [nvarchar](50) NULL,
	[NorthAmericaSeason] [nvarchar](50) NULL,
	[AsiaSeason] [nvarchar](50) NULL
)
WITH
(
    LOCATION='/DimDate/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimEmployee
CREATE EXTERNAL TABLE [adls2].DimEmployee 
(
	[EmployeeKey] [int]  NOT NULL,
	[ParentEmployeeKey] [int] NULL,
	[FirstName] [nvarchar](50) NOT NULL,
	[LastName] [nvarchar](50) NOT NULL,
	[MiddleName] [nvarchar](50) NULL,
	[Title] [nvarchar](50) NULL,
	[HireDate] [datetime] NULL,
	[BirthDate] [datetime] NULL,
	[EmailAddress] [nvarchar](50) NULL,
	[Phone] [nvarchar](25) NULL,
	[MaritalStatus] [nchar](1) NULL,
	[EmergencyContactName] [nvarchar](50) NULL,
	[EmergencyContactPhone] [nvarchar](25) NULL,
	[SalariedFlag] [bit] NULL,
	[Gender] [nchar](1) NULL,
	[PayFrequency] [tinyint] NULL,
	[BaseRate] [money] NULL,
	[VacationHours] [smallint] NULL,
	[CurrentFlag] [bit] NOT NULL,
	[SalesPersonFlag] [bit] NOT NULL,
	[DepartmentName] [nvarchar](50) NULL,
	[StartDate] [datetime] NULL,
	[EndDate] [datetime] NULL,
	[Status] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimEmployee/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimEntity
CREATE EXTERNAL TABLE [adls2].DimEntity 
(
	[EntityKey] [int] NOT NULL,
	[EntityLabel] [nvarchar](100) NULL,
	[ParentEntityKey] [int] NULL,
	[ParentEntityLabel] [nvarchar](100) NULL,
	[EntityName] [nvarchar](50) NULL,
	[EntityDescription] [nvarchar](100) NULL,
	[EntityType] [nvarchar](100) NULL,
	[StartDate] [datetime] NULL,
	[EndDate] [datetime] NULL,
	[Status] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimEntity/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimGeography
CREATE EXTERNAL TABLE [adls2].DimGeography 
(
	[GeographyKey] [int] NOT NULL,
	[GeographyType] [nvarchar](50) NOT NULL,
	[ContinentName] [nvarchar](50) NOT NULL,
	[CityName] [nvarchar](100) NULL,
	[StateProvinceName] [nvarchar](100) NULL,
	[RegionCountryName] [nvarchar](100) NULL,
--	[Geometry] [geometry] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimGeography/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimMachine
CREATE EXTERNAL TABLE [adls2].DimMachine 
(
	[MachineKey] [int] NOT NULL,
	[MachineLabel] [nvarchar](100) NULL,
	[StoreKey] [int] NOT NULL,
	[MachineType] [nvarchar](50) NOT NULL,
	[MachineName] [nvarchar](100) NOT NULL,
	[MachineDescription] [nvarchar](200) NOT NULL,
	[VendorName] [nvarchar](50) NOT NULL,
	[MachineOS] [nvarchar](50) NOT NULL,
	[MachineSource] [nvarchar](100) NOT NULL,
	[MachineHardware] [nvarchar](100) NULL,
	[MachineSoftware] [nvarchar](100) NOT NULL,
	[Status] [nvarchar](50) NOT NULL,
	[ServiceStartDate] [datetime] NOT NULL,
	[DecommissionDate] [datetime] NULL,
	[LastModifiedDate] [datetime] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimMachine/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimOutage
CREATE EXTERNAL TABLE [adls2].DimOutage (
	[OutageKey] [int]  NOT NULL,
	[OutageLabel] [nvarchar](100) NOT NULL,
	[OutageName] [nvarchar](50) NOT NULL,
	[OutageDescription] [nvarchar](200) NOT NULL,
	[OutageType] [nvarchar](50) NOT NULL,
	[OutageTypeDescription] [nvarchar](200) NOT NULL,
	[OutageSubType] [nvarchar](50) NOT NULL,
	[OutageSubTypeDescription] [nvarchar](200) NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimOutage/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimProduct
CREATE EXTERNAL TABLE [adls2].DimProduct (
	[ProductKey] [int] NOT NULL,
	[ProductLabel] [nvarchar](255) NULL,
	[ProductName] [nvarchar](500) NULL,
	[ProductDescription] [nvarchar](400) NULL,
	[ProductSubcategoryKey] [int] NULL,
	[Manufacturer] [nvarchar](50) NULL,
	[BrandName] [nvarchar](50) NULL,
	[ClassID] [nvarchar](10) NULL,
	[ClassName] [nvarchar](20) NULL,
	[StyleID] [nvarchar](10) NULL,
	[StyleName] [nvarchar](20) NULL,
	[ColorID] [nvarchar](10) NULL,
	[ColorName] [nvarchar](20) NOT NULL,
	[Size] [nvarchar](50) NULL,
	[SizeRange] [nvarchar](50) NULL,
	[SizeUnitMeasureID] [nvarchar](20) NULL,
	[Weight] [float] NULL,
	[WeightUnitMeasureID] [nvarchar](20) NULL,
	[UnitOfMeasureID] [nvarchar](10) NULL,
	[UnitOfMeasureName] [nvarchar](40) NULL,
	[StockTypeID] [nvarchar](10) NULL,
	[StockTypeName] [nvarchar](40) NULL,
	[UnitCost] [money] NULL,
	[UnitPrice] [money] NULL,
	[AvailableForSaleDate] [datetime] NULL,
	[StopSaleDate] [datetime] NULL,
	[Status] [nvarchar](7) NULL,
	[ImageURL] [nvarchar](150) NULL,
	[ProductURL] [nvarchar](150) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimProduct/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimProductCategory
CREATE EXTERNAL TABLE [adls2].DimProductCategory (
	[ProductCategoryKey] [int]  NOT NULL,
	[ProductCategoryLabel] [nvarchar](100) NULL,
	[ProductCategoryName] [nvarchar](30) NOT NULL,
	[ProductCategoryDescription] [nvarchar](50) NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimProductCategory/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimProductSubcategory
CREATE EXTERNAL TABLE [adls2].DimProductSubcategory (
	[ProductSubcategoryKey] [int]  NOT NULL,
	[ProductSubcategoryLabel] [nvarchar](100) NULL,
	[ProductSubcategoryName] [nvarchar](50) NOT NULL,
	[ProductSubcategoryDescription] [nvarchar](100) NULL,
	[ProductCategoryKey] [int] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimProductSubcategory/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimPromotion
CREATE EXTERNAL TABLE [adls2].DimPromotion (
	[PromotionKey] [int]  NOT NULL,
	[PromotionLabel] [nvarchar](100) NULL,
	[PromotionName] [nvarchar](100) NULL,
	[PromotionDescription] [nvarchar](255) NULL,
	[DiscountPercent] [float] NULL,
	[PromotionType] [nvarchar](50) NULL,
	[PromotionCategory] [nvarchar](50) NULL,
	[StartDate] [datetime] NOT NULL,
	[EndDate] [datetime] NULL,
	[MinQuantity] [int] NULL,
	[MaxQuantity] [int] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimPromotion/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimSalesTerritory
CREATE EXTERNAL TABLE [adls2].DimSalesTerritory (
	[SalesTerritoryKey] [int]  NOT NULL,
	[GeographyKey] [int] NOT NULL,
	[SalesTerritoryLabel] [nvarchar](100) NULL,
	[SalesTerritoryName] [nvarchar](50) NOT NULL,
	[SalesTerritoryRegion] [nvarchar](50) NOT NULL,
	[SalesTerritoryCountry] [nvarchar](50) NOT NULL,
	[SalesTerritoryGroup] [nvarchar](50) NULL,
	[SalesTerritoryLevel] [nvarchar](10) NULL,
	[SalesTerritoryManager] [int] NULL,
	[StartDate] [datetime] NULL,
	[EndDate] [datetime] NULL,
	[Status] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimSalesTerritory/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimScenario
CREATE EXTERNAL TABLE [adls2].DimScenario (
	[ScenarioKey] [int] NOT NULL,
	[ScenarioLabel] [nvarchar](100) NOT NULL,
	[ScenarioName] [nvarchar](20) NULL,
	[ScenarioDescription] [nvarchar](50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimScenario/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--DimStore
CREATE EXTERNAL TABLE [adls2].DimStore 
(
	[StoreKey] [int] NOT NULL,
	[GeographyKey] [int] NOT NULL,
	[StoreManager] [int] NULL,
	[StoreType] [nvarchar](15) NULL,
	[StoreName] [nvarchar](100) NOT NULL,
	[StoreDescription] [nvarchar](300) NOT NULL,
	[Status] [nvarchar](20) NOT NULL,
	[OpenDate] [datetime] NOT NULL,
	[CloseDate] [datetime] NULL,
	[EntityKey] [int] NULL,
	[ZipCode] [nvarchar](20) NULL,
	[ZipCodeExtension] [nvarchar](10) NULL,
	[StorePhone] [nvarchar](15) NULL,
	[StoreFax] [nvarchar](14) NULL,
	[AddressLine1] [nvarchar](100) NULL,
	[AddressLine2] [nvarchar](100) NULL,
	[CloseReason] [nvarchar](20) NULL,
	[EmployeeCount] [int] NULL,
	[SellingAreaSize] [float] NULL,
	[LastRemodelDate] [datetime] NULL,
	[GeoLocation]	NVARCHAR(50)  NULL,
	[Geometry]		NVARCHAR(50) NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/DimStore/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactExchangeRate
CREATE EXTERNAL TABLE [adls2].FactExchangeRate 
(
	[ExchangeRateKey] [int]  NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[AverageRate] [float] NOT NULL,
	[EndOfDayRate] [float] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/FactExchangeRate/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactInventory
CREATE EXTERNAL TABLE [adls2].FactInventory (
	[InventoryKey] [int]  NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[ProductKey] [int] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[OnHandQuantity] [int] NOT NULL,
	[OnOrderQuantity] [int] NOT NULL,
	[SafetyStockQuantity] [int] NULL,
	[UnitCost] [money] NOT NULL,
	[DaysInStock] [int] NULL,
	[MinDayInStock] [int] NULL,
	[MaxDayInStock] [int] NULL,
	[Aging] [int] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/FactInventory/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactITMachine
CREATE EXTERNAL TABLE [adls2].FactITMachine (
	[ITMachinekey] [int] NOT NULL,
	[MachineKey] [int] NOT NULL,
	[Datekey] [datetime] NOT NULL,
	[CostAmount] [money] NULL,
	[CostType] [nvarchar](200) NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/FactITMachine/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactITSLA
CREATE EXTERNAL TABLE [adls2].FactITSLA 
(
	[ITSLAkey] [int]  NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[MachineKey] [int] NOT NULL,
	[OutageKey] [int] NOT NULL,
	[OutageStartTime] [datetime] NOT NULL,
	[OutageEndTime] [datetime] NOT NULL,
	[DownTime] [int] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/FactITSLA/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactOnlineSales
CREATE EXTERNAL TABLE [adls2].FactOnlineSales 
(
	[OnlineSalesKey] [int]  NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[ProductKey] [int] NOT NULL,
	[PromotionKey] [int] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[CustomerKey] [int] NOT NULL,
	[SalesOrderNumber] [nvarchar](20) NOT NULL,
	[SalesOrderLineNumber] [int] NULL,
	[SalesQuantity] [int] NOT NULL,
	[SalesAmount] [money] NOT NULL,
	[ReturnQuantity] [int] NOT NULL,
	[ReturnAmount] [money] NULL,
	[DiscountQuantity] [int] NULL,
	[DiscountAmount] [money] NULL,
	[TotalCost] [money] NOT NULL,
	[UnitCost] [money] NULL,
	[UnitPrice] [money] NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/FactOnlineSales/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactSales
CREATE EXTERNAL TABLE [adls2].FactSales 
(
	[SalesKey] [int]  NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[channelKey] [int] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[ProductKey] [int] NOT NULL,
	[PromotionKey] [int] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[UnitCost] [money] NOT NULL,
	[UnitPrice] [money] NOT NULL,
	[SalesQuantity] [int] NOT NULL,
	[ReturnQuantity] [int] NOT NULL,
	[ReturnAmount] [money] NULL,
	[DiscountQuantity] [int] NULL,
	[DiscountAmount] [money] NULL,
	[TotalCost] [money] NOT NULL,
	[SalesAmount] [money] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/FactSales/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactSalesQuota
CREATE EXTERNAL TABLE [adls2].FactSalesQuota (
	[SalesQuotaKey] [int]  NOT NULL,
	[ChannelKey] [int] NOT NULL,
	[StoreKey] [int] NOT NULL,
	[ProductKey] [int] NOT NULL,
	[DateKey] [datetime] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[ScenarioKey] [int] NOT NULL,
	[SalesQuantityQuota] [money] NOT NULL,
	[SalesAmountQuota] [money] NOT NULL,
	[GrossMarginQuota] [money] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/FactSalesQuota/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;
GO
--FactStrategyPlan
CREATE EXTERNAL TABLE [adls2].FactStrategyPlan 
(
	[StrategyPlanKey] [int]  NOT NULL,
	[Datekey] [datetime] NOT NULL,
	[EntityKey] [int] NOT NULL,
	[ScenarioKey] [int] NOT NULL,
	[AccountKey] [int] NOT NULL,
	[CurrencyKey] [int] NOT NULL,
	[ProductCategoryKey] [int] NULL,
	[Amount] [money] NOT NULL,
	[ETLLoadID] [int] NULL,
	[LoadDate] [datetime] NULL,
	[UpdateDate] [datetime] NULL
)
WITH
(
    LOCATION='/FactStrategyPlan/' 
,   DATA_SOURCE = AzureDataLakeStorageGen2
,   FILE_FORMAT = TextFileFormat
,   REJECT_TYPE = VALUE
,   REJECT_VALUE = 0
)
;

## Load the data with CTAS

In [47]:
CREATE SCHEMA [cso1];

In [48]:
CREATE TABLE [cso1].[DimAccount]            WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimAccount]             OPTION (LABEL = 'CTAS : Load [cso1].[DimAccount]             ');
CREATE TABLE [cso1].[DimChannel]            WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimChannel]             OPTION (LABEL = 'CTAS : Load [cso1].[DimChannel]             ');
CREATE TABLE [cso1].[DimCurrency]           WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimCurrency]            OPTION (LABEL = 'CTAS : Load [cso1].[DimCurrency]            ');
CREATE TABLE [cso1].[DimCustomer]           WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimCustomer]            OPTION (LABEL = 'CTAS : Load [cso1].[DimCustomer]            ');
CREATE TABLE [cso1].[DimDate]               WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimDate]                OPTION (LABEL = 'CTAS : Load [cso1].[DimDate]                ');
CREATE TABLE [cso1].[DimEmployee]           WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimEmployee]            OPTION (LABEL = 'CTAS : Load [cso1].[DimEmployee]            ');
CREATE TABLE [cso1].[DimEntity]             WITH (DISTRIBUTION = HASH([EntityKey]   ) ) AS SELECT * FROM [adls1].[DimEntity]              OPTION (LABEL = 'CTAS : Load [cso1].[DimEntity]              ');
CREATE TABLE [cso1].[DimGeography]          WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimGeography]           OPTION (LABEL = 'CTAS : Load [cso1].[DimGeography]           ');
CREATE TABLE [cso1].[DimMachine]            WITH (DISTRIBUTION = HASH([MachineKey]  ) ) AS SELECT * FROM [adls1].[DimMachine]             OPTION (LABEL = 'CTAS : Load [cso1].[DimMachine]             ');
CREATE TABLE [cso1].[DimOutage]             WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimOutage]              OPTION (LABEL = 'CTAS : Load [cso1].[DimOutage]              ');
CREATE TABLE [cso1].[DimProduct]            WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls1].[DimProduct]             OPTION (LABEL = 'CTAS : Load [cso1].[DimProduct]             ');
CREATE TABLE [cso1].[DimProductCategory]    WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimProductCategory]     OPTION (LABEL = 'CTAS : Load [cso1].[DimProductCategory]     ');
CREATE TABLE [cso1].[DimScenario]           WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimScenario]            OPTION (LABEL = 'CTAS : Load [cso1].[DimScenario]            ');
CREATE TABLE [cso1].[DimPromotion]          WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimPromotion]           OPTION (LABEL = 'CTAS : Load [cso1].[DimPromotion]           ');
CREATE TABLE [cso1].[DimProductSubcategory] WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimProductSubcategory]  OPTION (LABEL = 'CTAS : Load [cso1].[DimProductSubcategory]  ');
CREATE TABLE [cso1].[DimSalesTerritory]     WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimSalesTerritory]      OPTION (LABEL = 'CTAS : Load [cso1].[DimSalesTerritory]      ');
CREATE TABLE [cso1].[DimStore]              WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls1].[DimStore]               OPTION (LABEL = 'CTAS : Load [cso1].[DimStore]               ');
CREATE TABLE [cso1].[FactITMachine]         WITH (DISTRIBUTION = HASH([MachineKey]  ) ) AS SELECT * FROM [adls1].[FactITMachine]          OPTION (LABEL = 'CTAS : Load [cso1].[FactITMachine]          ');
CREATE TABLE [cso1].[FactITSLA]             WITH (DISTRIBUTION = HASH([MachineKey]  ) ) AS SELECT * FROM [adls1].[FactITSLA]              OPTION (LABEL = 'CTAS : Load [cso1].[FactITSLA]              ');
CREATE TABLE [cso1].[FactInventory]         WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls1].[FactInventory]          OPTION (LABEL = 'CTAS : Load [cso1].[FactInventory]          ');
CREATE TABLE [cso1].[FactOnlineSales]       WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls1].[FactOnlineSales]        OPTION (LABEL = 'CTAS : Load [cso1].[FactOnlineSales]        ');
CREATE TABLE [cso1].[FactSales]             WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls1].[FactSales]              OPTION (LABEL = 'CTAS : Load [cso1].[FactSales]              ');
CREATE TABLE [cso1].[FactSalesQuota]        WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls1].[FactSalesQuota]         OPTION (LABEL = 'CTAS : Load [cso1].[FactSalesQuota]         ');
CREATE TABLE [cso1].[FactStrategyPlan]      WITH (DISTRIBUTION = HASH([EntityKey])  )   AS SELECT * FROM [adls1].[FactStrategyPlan]       OPTION (LABEL = 'CTAS : Load [cso1].[FactStrategyPlan]       ');
CREATE TABLE [cso1].[FactExchangeRate]      WITH (DISTRIBUTION = HASH([ExchangeRateKey])  ) AS SELECT * FROM [adls1].[FactExchangeRate]    OPTION (LABEL = 'CTAS : Load [cso1].[FactExchangeRate]       ');

In [49]:
CREATE SCHEMA [cso2];

In [50]:
CREATE TABLE [cso2].[DimAccount]            WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimAccount]             OPTION (LABEL = 'CTAS : Load [cso2].[DimAccount]             ');
CREATE TABLE [cso2].[DimChannel]            WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimChannel]             OPTION (LABEL = 'CTAS : Load [cso2].[DimChannel]             ');
CREATE TABLE [cso2].[DimCurrency]           WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimCurrency]            OPTION (LABEL = 'CTAS : Load [cso2].[DimCurrency]            ');
CREATE TABLE [cso2].[DimCustomer]           WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimCustomer]            OPTION (LABEL = 'CTAS : Load [cso2].[DimCustomer]            ');
CREATE TABLE [cso2].[DimDate]               WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimDate]                OPTION (LABEL = 'CTAS : Load [cso2].[DimDate]                ');
CREATE TABLE [cso2].[DimEmployee]           WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimEmployee]            OPTION (LABEL = 'CTAS : Load [cso2].[DimEmployee]            ');
CREATE TABLE [cso2].[DimEntity]             WITH (DISTRIBUTION = HASH([EntityKey]   ) ) AS SELECT * FROM [adls2].[DimEntity]              OPTION (LABEL = 'CTAS : Load [cso2].[DimEntity]              ');
CREATE TABLE [cso2].[DimGeography]          WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimGeography]           OPTION (LABEL = 'CTAS : Load [cso2].[DimGeography]           ');
CREATE TABLE [cso2].[DimMachine]            WITH (DISTRIBUTION = HASH([MachineKey]  ) ) AS SELECT * FROM [adls2].[DimMachine]             OPTION (LABEL = 'CTAS : Load [cso2].[DimMachine]             ');
CREATE TABLE [cso2].[DimOutage]             WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimOutage]              OPTION (LABEL = 'CTAS : Load [cso2].[DimOutage]              ');
CREATE TABLE [cso2].[DimProduct]            WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls2].[DimProduct]             OPTION (LABEL = 'CTAS : Load [cso2].[DimProduct]             ');
CREATE TABLE [cso2].[DimProductCategory]    WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimProductCategory]     OPTION (LABEL = 'CTAS : Load [cso2].[DimProductCategory]     ');
CREATE TABLE [cso2].[DimScenario]           WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimScenario]            OPTION (LABEL = 'CTAS : Load [cso2].[DimScenario]            ');
CREATE TABLE [cso2].[DimPromotion]          WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimPromotion]           OPTION (LABEL = 'CTAS : Load [cso2].[DimPromotion]           ');
CREATE TABLE [cso2].[DimProductSubcategory] WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimProductSubcategory]  OPTION (LABEL = 'CTAS : Load [cso2].[DimProductSubcategory]  ');
CREATE TABLE [cso2].[DimSalesTerritory]     WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimSalesTerritory]      OPTION (LABEL = 'CTAS : Load [cso2].[DimSalesTerritory]      ');
CREATE TABLE [cso2].[DimStore]              WITH (DISTRIBUTION = REPLICATE          )   AS SELECT * FROM [adls2].[DimStore]               OPTION (LABEL = 'CTAS : Load [cso2].[DimStore]               ');
CREATE TABLE [cso2].[FactITMachine]         WITH (DISTRIBUTION = HASH([MachineKey]  ) ) AS SELECT * FROM [adls2].[FactITMachine]          OPTION (LABEL = 'CTAS : Load [cso2].[FactITMachine]          ');
CREATE TABLE [cso2].[FactITSLA]             WITH (DISTRIBUTION = HASH([MachineKey]  ) ) AS SELECT * FROM [adls2].[FactITSLA]              OPTION (LABEL = 'CTAS : Load [cso2].[FactITSLA]              ');
CREATE TABLE [cso2].[FactInventory]         WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls2].[FactInventory]          OPTION (LABEL = 'CTAS : Load [cso2].[FactInventory]          ');
CREATE TABLE [cso2].[FactOnlineSales]       WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls2].[FactOnlineSales]        OPTION (LABEL = 'CTAS : Load [cso2].[FactOnlineSales]        ');
CREATE TABLE [cso2].[FactSales]             WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls2].[FactSales]              OPTION (LABEL = 'CTAS : Load [cso2].[FactSales]              ');
CREATE TABLE [cso2].[FactSalesQuota]        WITH (DISTRIBUTION = HASH([ProductKey]  ) ) AS SELECT * FROM [adls2].[FactSalesQuota]         OPTION (LABEL = 'CTAS : Load [cso2].[FactSalesQuota]         ');
CREATE TABLE [cso2].[FactStrategyPlan]      WITH (DISTRIBUTION = HASH([EntityKey])  )   AS SELECT * FROM [adls2].[FactStrategyPlan]       OPTION (LABEL = 'CTAS : Load [cso2].[FactStrategyPlan]       ');
CREATE TABLE [cso2].[FactExchangeRate]      WITH (DISTRIBUTION = HASH([ExchangeRateKey])  ) AS SELECT * FROM [adls2].[FactExchangeRate]    OPTION (LABEL = 'CTAS : Load [cso2].[FactExchangeRate]       ');

## Optimize columnstore compression

In [34]:
ALTER INDEX ALL ON [cso1].[FactStrategyPlan]         REBUILD;
ALTER INDEX ALL ON [cso1].[DimAccount]               REBUILD;
ALTER INDEX ALL ON [cso1].[DimChannel]               REBUILD;
ALTER INDEX ALL ON [cso1].[DimCurrency]              REBUILD;
ALTER INDEX ALL ON [cso1].[DimCustomer]              REBUILD;
ALTER INDEX ALL ON [cso1].[DimDate]                  REBUILD;
ALTER INDEX ALL ON [cso1].[DimEmployee]              REBUILD;
ALTER INDEX ALL ON [cso1].[DimEntity]                REBUILD;
ALTER INDEX ALL ON [cso1].[DimGeography]             REBUILD;
ALTER INDEX ALL ON [cso1].[DimMachine]               REBUILD;
ALTER INDEX ALL ON [cso1].[DimOutage]                REBUILD;
ALTER INDEX ALL ON [cso1].[DimProduct]               REBUILD;
ALTER INDEX ALL ON [cso1].[DimProductCategory]       REBUILD;
ALTER INDEX ALL ON [cso1].[DimScenario]              REBUILD;
ALTER INDEX ALL ON [cso1].[DimPromotion]             REBUILD;
ALTER INDEX ALL ON [cso1].[DimProductSubcategory]    REBUILD;
ALTER INDEX ALL ON [cso1].[DimSalesTerritory]        REBUILD;
ALTER INDEX ALL ON [cso1].[DimStore]                 REBUILD;
ALTER INDEX ALL ON [cso1].[FactITMachine]            REBUILD;
ALTER INDEX ALL ON [cso1].[FactITSLA]                REBUILD;
ALTER INDEX ALL ON [cso1].[FactInventory]            REBUILD;
ALTER INDEX ALL ON [cso1].[FactOnlineSales]          REBUILD;
ALTER INDEX ALL ON [cso1].[FactSales]                REBUILD;
ALTER INDEX ALL ON [cso1].[FactSalesQuota]           REBUILD;
ALTER INDEX ALL ON [cso1].[FactExchangeRate]         REBUILD;

In [35]:
ALTER INDEX ALL ON [cso2].[FactStrategyPlan]         REBUILD;
ALTER INDEX ALL ON [cso2].[DimAccount]               REBUILD;
ALTER INDEX ALL ON [cso2].[DimChannel]               REBUILD;
ALTER INDEX ALL ON [cso2].[DimCurrency]              REBUILD;
ALTER INDEX ALL ON [cso2].[DimCustomer]              REBUILD;
ALTER INDEX ALL ON [cso2].[DimDate]                  REBUILD;
ALTER INDEX ALL ON [cso2].[DimEmployee]              REBUILD;
ALTER INDEX ALL ON [cso2].[DimEntity]                REBUILD;
ALTER INDEX ALL ON [cso2].[DimGeography]             REBUILD;
ALTER INDEX ALL ON [cso2].[DimMachine]               REBUILD;
ALTER INDEX ALL ON [cso2].[DimOutage]                REBUILD;
ALTER INDEX ALL ON [cso2].[DimProduct]               REBUILD;
ALTER INDEX ALL ON [cso2].[DimProductCategory]       REBUILD;
ALTER INDEX ALL ON [cso2].[DimScenario]              REBUILD;
ALTER INDEX ALL ON [cso2].[DimPromotion]             REBUILD;
ALTER INDEX ALL ON [cso2].[DimProductSubcategory]    REBUILD;
ALTER INDEX ALL ON [cso2].[DimSalesTerritory]        REBUILD;
ALTER INDEX ALL ON [cso2].[DimStore]                 REBUILD;
ALTER INDEX ALL ON [cso2].[FactITMachine]            REBUILD;
ALTER INDEX ALL ON [cso2].[FactITSLA]                REBUILD;
ALTER INDEX ALL ON [cso2].[FactInventory]            REBUILD;
ALTER INDEX ALL ON [cso2].[FactOnlineSales]          REBUILD;
ALTER INDEX ALL ON [cso2].[FactSales]                REBUILD;
ALTER INDEX ALL ON [cso2].[FactSalesQuota]           REBUILD;
ALTER INDEX ALL ON [cso2].[FactExchangeRate]         REBUILD;

## Optimize statistics

In [36]:
CREATE STATISTICS [stat_cso1_DimMachine_DecommissionDate] ON [cso1].[DimMachine]([DecommissionDate]);
CREATE STATISTICS [stat_cso1_DimMachine_ETLLoadID] ON [cso1].[DimMachine]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimMachine_LastModifiedDate] ON [cso1].[DimMachine]([LastModifiedDate]);
CREATE STATISTICS [stat_cso1_DimMachine_LoadDate] ON [cso1].[DimMachine]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineDescription] ON [cso1].[DimMachine]([MachineDescription]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineHardware] ON [cso1].[DimMachine]([MachineHardware]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineKey] ON [cso1].[DimMachine]([MachineKey]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineLabel] ON [cso1].[DimMachine]([MachineLabel]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineName] ON [cso1].[DimMachine]([MachineName]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineOS] ON [cso1].[DimMachine]([MachineOS]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineSoftware] ON [cso1].[DimMachine]([MachineSoftware]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineSource] ON [cso1].[DimMachine]([MachineSource]);
CREATE STATISTICS [stat_cso1_DimMachine_MachineType] ON [cso1].[DimMachine]([MachineType]);
CREATE STATISTICS [stat_cso1_DimMachine_ServiceStartDate] ON [cso1].[DimMachine]([ServiceStartDate]);
CREATE STATISTICS [stat_cso1_DimMachine_Status] ON [cso1].[DimMachine]([Status]);
CREATE STATISTICS [stat_cso1_DimMachine_StoreKey] ON [cso1].[DimMachine]([StoreKey]);
CREATE STATISTICS [stat_cso1_DimMachine_UpdateDate] ON [cso1].[DimMachine]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimMachine_VendorName] ON [cso1].[DimMachine]([VendorName]);
CREATE STATISTICS [stat_cso1_DimOutage_ETLLoadID] ON [cso1].[DimOutage]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimOutage_LoadDate] ON [cso1].[DimOutage]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimOutage_OutageDescription] ON [cso1].[DimOutage]([OutageDescription]);
CREATE STATISTICS [stat_cso1_DimOutage_OutageKey] ON [cso1].[DimOutage]([OutageKey]);
CREATE STATISTICS [stat_cso1_DimOutage_OutageLabel] ON [cso1].[DimOutage]([OutageLabel]);
CREATE STATISTICS [stat_cso1_DimOutage_OutageName] ON [cso1].[DimOutage]([OutageName]);
CREATE STATISTICS [stat_cso1_DimOutage_OutageSubType] ON [cso1].[DimOutage]([OutageSubType]);
CREATE STATISTICS [stat_cso1_DimOutage_OutageSubTypeDescription] ON [cso1].[DimOutage]([OutageSubTypeDescription]);
CREATE STATISTICS [stat_cso1_DimOutage_OutageType] ON [cso1].[DimOutage]([OutageType]);
CREATE STATISTICS [stat_cso1_DimOutage_OutageTypeDescription] ON [cso1].[DimOutage]([OutageTypeDescription]);
CREATE STATISTICS [stat_cso1_DimOutage_UpdateDate] ON [cso1].[DimOutage]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimProductCategory_ETLLoadID] ON [cso1].[DimProductCategory]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimProductCategory_LoadDate] ON [cso1].[DimProductCategory]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimProductCategory_ProductCategoryDescription] ON [cso1].[DimProductCategory]([ProductCategoryDescription]);
CREATE STATISTICS [stat_cso1_DimProductCategory_ProductCategoryKey] ON [cso1].[DimProductCategory]([ProductCategoryKey]);
CREATE STATISTICS [stat_cso1_DimProductCategory_ProductCategoryLabel] ON [cso1].[DimProductCategory]([ProductCategoryLabel]);
CREATE STATISTICS [stat_cso1_DimProductCategory_ProductCategoryName] ON [cso1].[DimProductCategory]([ProductCategoryName]);
CREATE STATISTICS [stat_cso1_DimProductCategory_UpdateDate] ON [cso1].[DimProductCategory]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimScenario_ETLLoadID] ON [cso1].[DimScenario]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimScenario_LoadDate] ON [cso1].[DimScenario]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimScenario_ScenarioDescription] ON [cso1].[DimScenario]([ScenarioDescription]);
CREATE STATISTICS [stat_cso1_DimScenario_ScenarioKey] ON [cso1].[DimScenario]([ScenarioKey]);
CREATE STATISTICS [stat_cso1_DimScenario_ScenarioLabel] ON [cso1].[DimScenario]([ScenarioLabel]);
CREATE STATISTICS [stat_cso1_DimScenario_ScenarioName] ON [cso1].[DimScenario]([ScenarioName]);
CREATE STATISTICS [stat_cso1_DimScenario_UpdateDate] ON [cso1].[DimScenario]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimPromotion_DiscountPercent] ON [cso1].[DimPromotion]([DiscountPercent]);
CREATE STATISTICS [stat_cso1_DimPromotion_EndDate] ON [cso1].[DimPromotion]([EndDate]);
CREATE STATISTICS [stat_cso1_DimPromotion_ETLLoadID] ON [cso1].[DimPromotion]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimPromotion_LoadDate] ON [cso1].[DimPromotion]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimPromotion_MaxQuantity] ON [cso1].[DimPromotion]([MaxQuantity]);
CREATE STATISTICS [stat_cso1_DimPromotion_MinQuantity] ON [cso1].[DimPromotion]([MinQuantity]);
CREATE STATISTICS [stat_cso1_DimPromotion_PromotionCategory] ON [cso1].[DimPromotion]([PromotionCategory]);
CREATE STATISTICS [stat_cso1_DimPromotion_PromotionDescription] ON [cso1].[DimPromotion]([PromotionDescription]);
CREATE STATISTICS [stat_cso1_DimPromotion_PromotionKey] ON [cso1].[DimPromotion]([PromotionKey]);
CREATE STATISTICS [stat_cso1_DimPromotion_PromotionLabel] ON [cso1].[DimPromotion]([PromotionLabel]);
CREATE STATISTICS [stat_cso1_DimPromotion_PromotionName] ON [cso1].[DimPromotion]([PromotionName]);
CREATE STATISTICS [stat_cso1_DimPromotion_PromotionType] ON [cso1].[DimPromotion]([PromotionType]);
CREATE STATISTICS [stat_cso1_DimPromotion_StartDate] ON [cso1].[DimPromotion]([StartDate]);
CREATE STATISTICS [stat_cso1_DimPromotion_UpdateDate] ON [cso1].[DimPromotion]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_EndDate] ON [cso1].[DimSalesTerritory]([EndDate]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_ETLLoadID] ON [cso1].[DimSalesTerritory]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_GeographyKey] ON [cso1].[DimSalesTerritory]([GeographyKey]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_LoadDate] ON [cso1].[DimSalesTerritory]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_SalesTerritoryCountry] ON [cso1].[DimSalesTerritory]([SalesTerritoryCountry]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_SalesTerritoryGroup] ON [cso1].[DimSalesTerritory]([SalesTerritoryGroup]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_SalesTerritoryKey] ON [cso1].[DimSalesTerritory]([SalesTerritoryKey]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_SalesTerritoryLabel] ON [cso1].[DimSalesTerritory]([SalesTerritoryLabel]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_SalesTerritoryLevel] ON [cso1].[DimSalesTerritory]([SalesTerritoryLevel]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_SalesTerritoryManager] ON [cso1].[DimSalesTerritory]([SalesTerritoryManager]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_SalesTerritoryName] ON [cso1].[DimSalesTerritory]([SalesTerritoryName]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_SalesTerritoryRegion] ON [cso1].[DimSalesTerritory]([SalesTerritoryRegion]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_StartDate] ON [cso1].[DimSalesTerritory]([StartDate]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_Status] ON [cso1].[DimSalesTerritory]([Status]);
CREATE STATISTICS [stat_cso1_DimSalesTerritory_UpdateDate] ON [cso1].[DimSalesTerritory]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimProductSubcategory_ETLLoadID] ON [cso1].[DimProductSubcategory]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimProductSubcategory_LoadDate] ON [cso1].[DimProductSubcategory]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimProductSubcategory_ProductCategoryKey] ON [cso1].[DimProductSubcategory]([ProductCategoryKey]);
CREATE STATISTICS [stat_cso1_DimProductSubcategory_ProductSubcategoryDescription] ON [cso1].[DimProductSubcategory]([ProductSubcategoryDescription]);
CREATE STATISTICS [stat_cso1_DimProductSubcategory_ProductSubcategoryKey] ON [cso1].[DimProductSubcategory]([ProductSubcategoryKey]);
CREATE STATISTICS [stat_cso1_DimProductSubcategory_ProductSubcategoryLabel] ON [cso1].[DimProductSubcategory]([ProductSubcategoryLabel]);
CREATE STATISTICS [stat_cso1_DimProductSubcategory_ProductSubcategoryName] ON [cso1].[DimProductSubcategory]([ProductSubcategoryName]);
CREATE STATISTICS [stat_cso1_DimProductSubcategory_UpdateDate] ON [cso1].[DimProductSubcategory]([UpdateDate]);
CREATE STATISTICS [stat_cso1_FactITMachine_Datekey] ON [cso1].[FactITMachine]([Datekey]);
CREATE STATISTICS [stat_cso1_FactITMachine_ITMachinekey] ON [cso1].[FactITMachine]([ITMachinekey]);
CREATE STATISTICS [stat_cso1_FactITMachine_MachineKey] ON [cso1].[FactITMachine]([MachineKey]);
CREATE STATISTICS [stat_cso1_FactInventory_CurrencyKey] ON [cso1].[FactInventory]([CurrencyKey]);
CREATE STATISTICS [stat_cso1_FactInventory_DateKey] ON [cso1].[FactInventory]([DateKey]);
CREATE STATISTICS [stat_cso1_FactInventory_InventoryKey] ON [cso1].[FactInventory]([InventoryKey]);
CREATE STATISTICS [stat_cso1_FactInventory_ProductKey] ON [cso1].[FactInventory]([ProductKey]);
CREATE STATISTICS [stat_cso1_FactInventory_StoreKey] ON [cso1].[FactInventory]([StoreKey]);
CREATE STATISTICS [stat_cso1_FactStrategyPlan_AccountKey] ON [cso1].[FactStrategyPlan]([AccountKey]);
CREATE STATISTICS [stat_cso1_FactStrategyPlan_CurrencyKey] ON [cso1].[FactStrategyPlan]([CurrencyKey]);
CREATE STATISTICS [stat_cso1_FactStrategyPlan_Datekey] ON [cso1].[FactStrategyPlan]([Datekey]);
CREATE STATISTICS [stat_cso1_FactStrategyPlan_EntityKey] ON [cso1].[FactStrategyPlan]([EntityKey]);
CREATE STATISTICS [stat_cso1_FactStrategyPlan_ProductCategoryKey] ON [cso1].[FactStrategyPlan]([ProductCategoryKey]);
CREATE STATISTICS [stat_cso1_FactStrategyPlan_ScenarioKey] ON [cso1].[FactStrategyPlan]([ScenarioKey]);
CREATE STATISTICS [stat_cso1_FactStrategyPlan_StrategyPlanKey] ON [cso1].[FactStrategyPlan]([StrategyPlanKey]);
CREATE STATISTICS [stat_cso1_FactSalesQuota_ChannelKey] ON [cso1].[FactSalesQuota]([ChannelKey]);
CREATE STATISTICS [stat_cso1_FactSalesQuota_CurrencyKey] ON [cso1].[FactSalesQuota]([CurrencyKey]);
CREATE STATISTICS [stat_cso1_FactSalesQuota_DateKey] ON [cso1].[FactSalesQuota]([DateKey]);
CREATE STATISTICS [stat_cso1_FactSalesQuota_ProductKey] ON [cso1].[FactSalesQuota]([ProductKey]);
CREATE STATISTICS [stat_cso1_FactSalesQuota_SalesQuotaKey] ON [cso1].[FactSalesQuota]([SalesQuotaKey]);
CREATE STATISTICS [stat_cso1_FactSalesQuota_ScenarioKey] ON [cso1].[FactSalesQuota]([ScenarioKey]);
CREATE STATISTICS [stat_cso1_FactSalesQuota_StoreKey] ON [cso1].[FactSalesQuota]([StoreKey]);
CREATE STATISTICS [stat_cso1_FactSales_channelKey] ON [cso1].[FactSales]([channelKey]);
CREATE STATISTICS [stat_cso1_FactSales_CurrencyKey] ON [cso1].[FactSales]([CurrencyKey]);
CREATE STATISTICS [stat_cso1_FactSales_DateKey] ON [cso1].[FactSales]([DateKey]);
CREATE STATISTICS [stat_cso1_FactSales_ProductKey] ON [cso1].[FactSales]([ProductKey]);
CREATE STATISTICS [stat_cso1_FactSales_PromotionKey] ON [cso1].[FactSales]([PromotionKey]);
CREATE STATISTICS [stat_cso1_FactSales_SalesKey] ON [cso1].[FactSales]([SalesKey]);
CREATE STATISTICS [stat_cso1_FactSales_StoreKey] ON [cso1].[FactSales]([StoreKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_CurrencyKey] ON [cso1].[FactOnlineSales]([CurrencyKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_CustomerKey] ON [cso1].[FactOnlineSales]([CustomerKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_DateKey] ON [cso1].[FactOnlineSales]([DateKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_OnlineSalesKey] ON [cso1].[FactOnlineSales]([OnlineSalesKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_ProductKey] ON [cso1].[FactOnlineSales]([ProductKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_PromotionKey] ON [cso1].[FactOnlineSales]([PromotionKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_StoreKey] ON [cso1].[FactOnlineSales]([StoreKey]);
CREATE STATISTICS [stat_cso1_DimCustomer_AddressLine1] ON [cso1].[DimCustomer]([AddressLine1]);
CREATE STATISTICS [stat_cso1_DimCustomer_AddressLine2] ON [cso1].[DimCustomer]([AddressLine2]);
CREATE STATISTICS [stat_cso1_DimCustomer_BirthDate] ON [cso1].[DimCustomer]([BirthDate]);
CREATE STATISTICS [stat_cso1_DimCustomer_CompanyName] ON [cso1].[DimCustomer]([CompanyName]);
CREATE STATISTICS [stat_cso1_DimCustomer_CustomerKey] ON [cso1].[DimCustomer]([CustomerKey]);
CREATE STATISTICS [stat_cso1_DimCustomer_CustomerLabel] ON [cso1].[DimCustomer]([CustomerLabel]);
CREATE STATISTICS [stat_cso1_DimCustomer_CustomerType] ON [cso1].[DimCustomer]([CustomerType]);
CREATE STATISTICS [stat_cso1_DimCustomer_DateFirstPurchase] ON [cso1].[DimCustomer]([DateFirstPurchase]);
CREATE STATISTICS [stat_cso1_DimCustomer_Education] ON [cso1].[DimCustomer]([Education]);
CREATE STATISTICS [stat_cso1_DimCustomer_EmailAddress] ON [cso1].[DimCustomer]([EmailAddress]);
CREATE STATISTICS [stat_cso1_DimCustomer_ETLLoadID] ON [cso1].[DimCustomer]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimCustomer_FirstName] ON [cso1].[DimCustomer]([FirstName]);
CREATE STATISTICS [stat_cso1_DimCustomer_Gender] ON [cso1].[DimCustomer]([Gender]);
CREATE STATISTICS [stat_cso1_DimCustomer_GeographyKey] ON [cso1].[DimCustomer]([GeographyKey]);
CREATE STATISTICS [stat_cso1_DimCustomer_HouseOwnerFlag] ON [cso1].[DimCustomer]([HouseOwnerFlag]);
CREATE STATISTICS [stat_cso1_DimCustomer_LastName] ON [cso1].[DimCustomer]([LastName]);
CREATE STATISTICS [stat_cso1_DimCustomer_LoadDate] ON [cso1].[DimCustomer]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimCustomer_MaritalStatus] ON [cso1].[DimCustomer]([MaritalStatus]);
CREATE STATISTICS [stat_cso1_DimCustomer_MiddleName] ON [cso1].[DimCustomer]([MiddleName]);
CREATE STATISTICS [stat_cso1_DimCustomer_NameStyle] ON [cso1].[DimCustomer]([NameStyle]);
CREATE STATISTICS [stat_cso1_DimCustomer_NumberCarsOwned] ON [cso1].[DimCustomer]([NumberCarsOwned]);
CREATE STATISTICS [stat_cso1_DimCustomer_NumberChildrenAtHome] ON [cso1].[DimCustomer]([NumberChildrenAtHome]);
CREATE STATISTICS [stat_cso1_DimCustomer_Occupation] ON [cso1].[DimCustomer]([Occupation]);
CREATE STATISTICS [stat_cso1_DimCustomer_Phone] ON [cso1].[DimCustomer]([Phone]);
CREATE STATISTICS [stat_cso1_DimCustomer_Suffix] ON [cso1].[DimCustomer]([Suffix]);
CREATE STATISTICS [stat_cso1_DimCustomer_Title] ON [cso1].[DimCustomer]([Title]);
CREATE STATISTICS [stat_cso1_DimCustomer_TotalChildren] ON [cso1].[DimCustomer]([TotalChildren]);
CREATE STATISTICS [stat_cso1_DimCustomer_UpdateDate] ON [cso1].[DimCustomer]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimCustomer_YearlyIncome] ON [cso1].[DimCustomer]([YearlyIncome]);
CREATE STATISTICS [stat_cso1_DimEmployee_BaseRate] ON [cso1].[DimEmployee]([BaseRate]);
CREATE STATISTICS [stat_cso1_DimEmployee_BirthDate] ON [cso1].[DimEmployee]([BirthDate]);
CREATE STATISTICS [stat_cso1_DimEmployee_CurrentFlag] ON [cso1].[DimEmployee]([CurrentFlag]);
CREATE STATISTICS [stat_cso1_DimEmployee_DepartmentName] ON [cso1].[DimEmployee]([DepartmentName]);
CREATE STATISTICS [stat_cso1_DimEmployee_EmailAddress] ON [cso1].[DimEmployee]([EmailAddress]);
CREATE STATISTICS [stat_cso1_DimEmployee_EmergencyContactName] ON [cso1].[DimEmployee]([EmergencyContactName]);
CREATE STATISTICS [stat_cso1_DimEmployee_EmergencyContactPhone] ON [cso1].[DimEmployee]([EmergencyContactPhone]);
CREATE STATISTICS [stat_cso1_DimEmployee_EmployeeKey] ON [cso1].[DimEmployee]([EmployeeKey]);
CREATE STATISTICS [stat_cso1_DimEmployee_EndDate] ON [cso1].[DimEmployee]([EndDate]);
CREATE STATISTICS [stat_cso1_DimEmployee_ETLLoadID] ON [cso1].[DimEmployee]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimEmployee_FirstName] ON [cso1].[DimEmployee]([FirstName]);
CREATE STATISTICS [stat_cso1_DimEmployee_Gender] ON [cso1].[DimEmployee]([Gender]);
CREATE STATISTICS [stat_cso1_DimEmployee_HireDate] ON [cso1].[DimEmployee]([HireDate]);
CREATE STATISTICS [stat_cso1_DimEmployee_LastName] ON [cso1].[DimEmployee]([LastName]);
CREATE STATISTICS [stat_cso1_DimEmployee_LoadDate] ON [cso1].[DimEmployee]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimEmployee_MaritalStatus] ON [cso1].[DimEmployee]([MaritalStatus]);
CREATE STATISTICS [stat_cso1_DimEmployee_MiddleName] ON [cso1].[DimEmployee]([MiddleName]);
CREATE STATISTICS [stat_cso1_DimEmployee_ParentEmployeeKey] ON [cso1].[DimEmployee]([ParentEmployeeKey]);
CREATE STATISTICS [stat_cso1_DimEmployee_PayFrequency] ON [cso1].[DimEmployee]([PayFrequency]);
CREATE STATISTICS [stat_cso1_DimEmployee_Phone] ON [cso1].[DimEmployee]([Phone]);
CREATE STATISTICS [stat_cso1_DimEmployee_SalariedFlag] ON [cso1].[DimEmployee]([SalariedFlag]);
CREATE STATISTICS [stat_cso1_DimEmployee_SalesPersonFlag] ON [cso1].[DimEmployee]([SalesPersonFlag]);
CREATE STATISTICS [stat_cso1_DimEmployee_StartDate] ON [cso1].[DimEmployee]([StartDate]);
CREATE STATISTICS [stat_cso1_DimEmployee_Status] ON [cso1].[DimEmployee]([Status]);
CREATE STATISTICS [stat_cso1_DimEmployee_Title] ON [cso1].[DimEmployee]([Title]);
CREATE STATISTICS [stat_cso1_DimEmployee_UpdateDate] ON [cso1].[DimEmployee]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimEmployee_VacationHours] ON [cso1].[DimEmployee]([VacationHours]);
CREATE STATISTICS [stat_cso1_DimEntity_EndDate] ON [cso1].[DimEntity]([EndDate]);
CREATE STATISTICS [stat_cso1_DimEntity_EntityDescription] ON [cso1].[DimEntity]([EntityDescription]);
CREATE STATISTICS [stat_cso1_DimEntity_EntityKey] ON [cso1].[DimEntity]([EntityKey]);
CREATE STATISTICS [stat_cso1_DimEntity_EntityLabel] ON [cso1].[DimEntity]([EntityLabel]);
CREATE STATISTICS [stat_cso1_DimEntity_EntityName] ON [cso1].[DimEntity]([EntityName]);
CREATE STATISTICS [stat_cso1_DimEntity_EntityType] ON [cso1].[DimEntity]([EntityType]);
CREATE STATISTICS [stat_cso1_DimEntity_ETLLoadID] ON [cso1].[DimEntity]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimEntity_LoadDate] ON [cso1].[DimEntity]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimEntity_ParentEntityKey] ON [cso1].[DimEntity]([ParentEntityKey]);
CREATE STATISTICS [stat_cso1_DimEntity_ParentEntityLabel] ON [cso1].[DimEntity]([ParentEntityLabel]);
CREATE STATISTICS [stat_cso1_DimEntity_StartDate] ON [cso1].[DimEntity]([StartDate]);
CREATE STATISTICS [stat_cso1_DimEntity_Status] ON [cso1].[DimEntity]([Status]);
CREATE STATISTICS [stat_cso1_DimEntity_UpdateDate] ON [cso1].[DimEntity]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimProduct_AvailableForSaleDate] ON [cso1].[DimProduct]([AvailableForSaleDate]);
CREATE STATISTICS [stat_cso1_DimProduct_BrandName] ON [cso1].[DimProduct]([BrandName]);
CREATE STATISTICS [stat_cso1_DimProduct_ClassID] ON [cso1].[DimProduct]([ClassID]);
CREATE STATISTICS [stat_cso1_DimProduct_ClassName] ON [cso1].[DimProduct]([ClassName]);
CREATE STATISTICS [stat_cso1_DimProduct_ColorID] ON [cso1].[DimProduct]([ColorID]);
CREATE STATISTICS [stat_cso1_DimProduct_ColorName] ON [cso1].[DimProduct]([ColorName]);
CREATE STATISTICS [stat_cso1_DimProduct_ETLLoadID] ON [cso1].[DimProduct]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimProduct_ImageURL] ON [cso1].[DimProduct]([ImageURL]);
CREATE STATISTICS [stat_cso1_DimProduct_LoadDate] ON [cso1].[DimProduct]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimProduct_Manufacturer] ON [cso1].[DimProduct]([Manufacturer]);
CREATE STATISTICS [stat_cso1_DimProduct_ProductDescription] ON [cso1].[DimProduct]([ProductDescription]);
CREATE STATISTICS [stat_cso1_DimProduct_ProductKey] ON [cso1].[DimProduct]([ProductKey]);
CREATE STATISTICS [stat_cso1_DimProduct_ProductLabel] ON [cso1].[DimProduct]([ProductLabel]);
CREATE STATISTICS [stat_cso1_DimProduct_ProductName] ON [cso1].[DimProduct]([ProductName]);
CREATE STATISTICS [stat_cso1_DimProduct_ProductSubcategoryKey] ON [cso1].[DimProduct]([ProductSubcategoryKey]);
CREATE STATISTICS [stat_cso1_DimProduct_ProductURL] ON [cso1].[DimProduct]([ProductURL]);
CREATE STATISTICS [stat_cso1_DimProduct_Size] ON [cso1].[DimProduct]([Size]);
CREATE STATISTICS [stat_cso1_DimProduct_SizeRange] ON [cso1].[DimProduct]([SizeRange]);
CREATE STATISTICS [stat_cso1_DimProduct_SizeUnitMeasureID] ON [cso1].[DimProduct]([SizeUnitMeasureID]);
CREATE STATISTICS [stat_cso1_DimProduct_Status] ON [cso1].[DimProduct]([Status]);
CREATE STATISTICS [stat_cso1_DimProduct_StockTypeID] ON [cso1].[DimProduct]([StockTypeID]);
CREATE STATISTICS [stat_cso1_DimProduct_StockTypeName] ON [cso1].[DimProduct]([StockTypeName]);
CREATE STATISTICS [stat_cso1_DimProduct_StopSaleDate] ON [cso1].[DimProduct]([StopSaleDate]);
CREATE STATISTICS [stat_cso1_DimProduct_StyleID] ON [cso1].[DimProduct]([StyleID]);
CREATE STATISTICS [stat_cso1_DimProduct_StyleName] ON [cso1].[DimProduct]([StyleName]);
CREATE STATISTICS [stat_cso1_DimProduct_UnitCost] ON [cso1].[DimProduct]([UnitCost]);
CREATE STATISTICS [stat_cso1_DimProduct_UnitOfMeasureID] ON [cso1].[DimProduct]([UnitOfMeasureID]);
CREATE STATISTICS [stat_cso1_DimProduct_UnitOfMeasureName] ON [cso1].[DimProduct]([UnitOfMeasureName]);
CREATE STATISTICS [stat_cso1_DimProduct_UnitPrice] ON [cso1].[DimProduct]([UnitPrice]);
CREATE STATISTICS [stat_cso1_DimProduct_UpdateDate] ON [cso1].[DimProduct]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimProduct_Weight] ON [cso1].[DimProduct]([Weight]);
CREATE STATISTICS [stat_cso1_DimProduct_WeightUnitMeasureID] ON [cso1].[DimProduct]([WeightUnitMeasureID]);
CREATE STATISTICS [stat_cso1_DimAccount_AccountDescription] ON [cso1].[DimAccount]([AccountDescription]);
CREATE STATISTICS [stat_cso1_DimAccount_AccountKey] ON [cso1].[DimAccount]([AccountKey]);
CREATE STATISTICS [stat_cso1_DimAccount_AccountLabel] ON [cso1].[DimAccount]([AccountLabel]);
CREATE STATISTICS [stat_cso1_DimAccount_AccountName] ON [cso1].[DimAccount]([AccountName]);
CREATE STATISTICS [stat_cso1_DimAccount_AccountType] ON [cso1].[DimAccount]([AccountType]);
CREATE STATISTICS [stat_cso1_DimAccount_CustomMemberOptions] ON [cso1].[DimAccount]([CustomMemberOptions]);
CREATE STATISTICS [stat_cso1_DimAccount_CustomMembers] ON [cso1].[DimAccount]([CustomMembers]);
CREATE STATISTICS [stat_cso1_DimAccount_ETLLoadID] ON [cso1].[DimAccount]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimAccount_LoadDate] ON [cso1].[DimAccount]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimAccount_Operator] ON [cso1].[DimAccount]([Operator]);
CREATE STATISTICS [stat_cso1_DimAccount_ParentAccountKey] ON [cso1].[DimAccount]([ParentAccountKey]);
CREATE STATISTICS [stat_cso1_DimAccount_UpdateDate] ON [cso1].[DimAccount]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimAccount_ValueType] ON [cso1].[DimAccount]([ValueType]);
CREATE STATISTICS [stat_cso1_DimChannel_ChannelDescription] ON [cso1].[DimChannel]([ChannelDescription]);
CREATE STATISTICS [stat_cso1_DimChannel_ChannelKey] ON [cso1].[DimChannel]([ChannelKey]);
CREATE STATISTICS [stat_cso1_DimChannel_ChannelLabel] ON [cso1].[DimChannel]([ChannelLabel]);
CREATE STATISTICS [stat_cso1_DimChannel_ChannelName] ON [cso1].[DimChannel]([ChannelName]);
CREATE STATISTICS [stat_cso1_DimChannel_ETLLoadID] ON [cso1].[DimChannel]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimChannel_LoadDate] ON [cso1].[DimChannel]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimChannel_UpdateDate] ON [cso1].[DimChannel]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimCurrency_CurrencyDescription] ON [cso1].[DimCurrency]([CurrencyDescription]);
CREATE STATISTICS [stat_cso1_DimCurrency_CurrencyKey] ON [cso1].[DimCurrency]([CurrencyKey]);
CREATE STATISTICS [stat_cso1_DimCurrency_CurrencyLabel] ON [cso1].[DimCurrency]([CurrencyLabel]);
CREATE STATISTICS [stat_cso1_DimCurrency_CurrencyName] ON [cso1].[DimCurrency]([CurrencyName]);
CREATE STATISTICS [stat_cso1_DimCurrency_ETLLoadID] ON [cso1].[DimCurrency]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimCurrency_LoadDate] ON [cso1].[DimCurrency]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimCurrency_UpdateDate] ON [cso1].[DimCurrency]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimDate_AsiaSeason] ON [cso1].[DimDate]([AsiaSeason]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarDayOfWeek] ON [cso1].[DimDate]([CalendarDayOfWeek]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarDayOfWeekLabel] ON [cso1].[DimDate]([CalendarDayOfWeekLabel]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarHalfYear] ON [cso1].[DimDate]([CalendarHalfYear]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarHalfYearLabel] ON [cso1].[DimDate]([CalendarHalfYearLabel]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarMonth] ON [cso1].[DimDate]([CalendarMonth]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarMonthLabel] ON [cso1].[DimDate]([CalendarMonthLabel]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarQuarter] ON [cso1].[DimDate]([CalendarQuarter]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarQuarterLabel] ON [cso1].[DimDate]([CalendarQuarterLabel]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarWeek] ON [cso1].[DimDate]([CalendarWeek]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarWeekLabel] ON [cso1].[DimDate]([CalendarWeekLabel]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarYear] ON [cso1].[DimDate]([CalendarYear]);
CREATE STATISTICS [stat_cso1_DimDate_CalendarYearLabel] ON [cso1].[DimDate]([CalendarYearLabel]);
CREATE STATISTICS [stat_cso1_DimDate_DateDescription] ON [cso1].[DimDate]([DateDescription]);
CREATE STATISTICS [stat_cso1_DimDate_Datekey] ON [cso1].[DimDate]([Datekey]);
CREATE STATISTICS [stat_cso1_DimDate_EuropeSeason] ON [cso1].[DimDate]([EuropeSeason]);
CREATE STATISTICS [stat_cso1_DimDate_FiscalHalfYear] ON [cso1].[DimDate]([FiscalHalfYear]);
CREATE STATISTICS [stat_cso1_DimDate_FiscalHalfYearLabel] ON [cso1].[DimDate]([FiscalHalfYearLabel]);
CREATE STATISTICS [stat_cso1_DimDate_FiscalMonth] ON [cso1].[DimDate]([FiscalMonth]);
CREATE STATISTICS [stat_cso1_DimDate_FiscalMonthLabel] ON [cso1].[DimDate]([FiscalMonthLabel]);
CREATE STATISTICS [stat_cso1_DimDate_FiscalQuarter] ON [cso1].[DimDate]([FiscalQuarter]);
CREATE STATISTICS [stat_cso1_DimDate_FiscalQuarterLabel] ON [cso1].[DimDate]([FiscalQuarterLabel]);
CREATE STATISTICS [stat_cso1_DimDate_FiscalYear] ON [cso1].[DimDate]([FiscalYear]);
CREATE STATISTICS [stat_cso1_DimDate_FiscalYearLabel] ON [cso1].[DimDate]([FiscalYearLabel]);
CREATE STATISTICS [stat_cso1_DimDate_FullDateLabel] ON [cso1].[DimDate]([FullDateLabel]);
CREATE STATISTICS [stat_cso1_DimDate_HolidayName] ON [cso1].[DimDate]([HolidayName]);
CREATE STATISTICS [stat_cso1_DimDate_IsHoliday] ON [cso1].[DimDate]([IsHoliday]);
CREATE STATISTICS [stat_cso1_DimDate_IsWorkDay] ON [cso1].[DimDate]([IsWorkDay]);
CREATE STATISTICS [stat_cso1_DimDate_NorthAmericaSeason] ON [cso1].[DimDate]([NorthAmericaSeason]);
CREATE STATISTICS [stat_cso1_DimStore_AddressLine1] ON [cso1].[DimStore]([AddressLine1]);
CREATE STATISTICS [stat_cso1_DimStore_AddressLine2] ON [cso1].[DimStore]([AddressLine2]);
CREATE STATISTICS [stat_cso1_DimStore_CloseDate] ON [cso1].[DimStore]([CloseDate]);
CREATE STATISTICS [stat_cso1_DimStore_CloseReason] ON [cso1].[DimStore]([CloseReason]);
CREATE STATISTICS [stat_cso1_DimStore_EmployeeCount] ON [cso1].[DimStore]([EmployeeCount]);
CREATE STATISTICS [stat_cso1_DimStore_EntityKey] ON [cso1].[DimStore]([EntityKey]);
CREATE STATISTICS [stat_cso1_DimStore_ETLLoadID] ON [cso1].[DimStore]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimStore_GeographyKey] ON [cso1].[DimStore]([GeographyKey]);
CREATE STATISTICS [stat_cso1_DimStore_GeoLocation] ON [cso1].[DimStore]([GeoLocation]);
CREATE STATISTICS [stat_cso1_DimStore_Geometry] ON [cso1].[DimStore]([Geometry]);
CREATE STATISTICS [stat_cso1_DimStore_LastRemodelDate] ON [cso1].[DimStore]([LastRemodelDate]);
CREATE STATISTICS [stat_cso1_DimStore_LoadDate] ON [cso1].[DimStore]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimStore_OpenDate] ON [cso1].[DimStore]([OpenDate]);
CREATE STATISTICS [stat_cso1_DimStore_SellingAreaSize] ON [cso1].[DimStore]([SellingAreaSize]);
CREATE STATISTICS [stat_cso1_DimStore_Status] ON [cso1].[DimStore]([Status]);
CREATE STATISTICS [stat_cso1_DimStore_StoreDescription] ON [cso1].[DimStore]([StoreDescription]);
CREATE STATISTICS [stat_cso1_DimStore_StoreFax] ON [cso1].[DimStore]([StoreFax]);
CREATE STATISTICS [stat_cso1_DimStore_StoreKey] ON [cso1].[DimStore]([StoreKey]);
CREATE STATISTICS [stat_cso1_DimStore_StoreManager] ON [cso1].[DimStore]([StoreManager]);
CREATE STATISTICS [stat_cso1_DimStore_StoreName] ON [cso1].[DimStore]([StoreName]);
CREATE STATISTICS [stat_cso1_DimStore_StorePhone] ON [cso1].[DimStore]([StorePhone]);
CREATE STATISTICS [stat_cso1_DimStore_StoreType] ON [cso1].[DimStore]([StoreType]);
CREATE STATISTICS [stat_cso1_DimStore_UpdateDate] ON [cso1].[DimStore]([UpdateDate]);
CREATE STATISTICS [stat_cso1_DimStore_ZipCode] ON [cso1].[DimStore]([ZipCode]);
CREATE STATISTICS [stat_cso1_DimStore_ZipCodeExtension] ON [cso1].[DimStore]([ZipCodeExtension]);
CREATE STATISTICS [stat_cso1_DimGeography_CityName] ON [cso1].[DimGeography]([CityName]);
CREATE STATISTICS [stat_cso1_DimGeography_ContinentName] ON [cso1].[DimGeography]([ContinentName]);
CREATE STATISTICS [stat_cso1_DimGeography_ETLLoadID] ON [cso1].[DimGeography]([ETLLoadID]);
CREATE STATISTICS [stat_cso1_DimGeography_GeographyKey] ON [cso1].[DimGeography]([GeographyKey]);
CREATE STATISTICS [stat_cso1_DimGeography_GeographyType] ON [cso1].[DimGeography]([GeographyType]);
CREATE STATISTICS [stat_cso1_DimGeography_LoadDate] ON [cso1].[DimGeography]([LoadDate]);
CREATE STATISTICS [stat_cso1_DimGeography_RegionCountryName] ON [cso1].[DimGeography]([RegionCountryName]);
CREATE STATISTICS [stat_cso1_DimGeography_StateProvinceName] ON [cso1].[DimGeography]([StateProvinceName]);
CREATE STATISTICS [stat_cso1_DimGeography_UpdateDate] ON [cso1].[DimGeography]([UpdateDate]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_CurrencyKey] ON [cso1].[FactOnlineSales]([CurrencyKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_CustomerKey] ON [cso1].[FactOnlineSales]([CustomerKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_DateKey] ON [cso1].[FactOnlineSales]([DateKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_OnlineSalesKey] ON [cso1].[FactOnlineSales]([OnlineSalesKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_ProductKey] ON [cso1].[FactOnlineSales]([ProductKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_PromotionKey] ON [cso1].[FactOnlineSales]([PromotionKey]);
CREATE STATISTICS [stat_cso1_FactOnlineSales_StoreKey] ON [cso1].[FactOnlineSales]([StoreKey]);

In [0]:
CREATE STATISTICS [stat_cso2_DimMachine_DecommissionDate] ON [cso2].[DimMachine]([DecommissionDate]);
CREATE STATISTICS [stat_cso2_DimMachine_ETLLoadID] ON [cso2].[DimMachine]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimMachine_LastModifiedDate] ON [cso2].[DimMachine]([LastModifiedDate]);
CREATE STATISTICS [stat_cso2_DimMachine_LoadDate] ON [cso2].[DimMachine]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineDescription] ON [cso2].[DimMachine]([MachineDescription]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineHardware] ON [cso2].[DimMachine]([MachineHardware]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineKey] ON [cso2].[DimMachine]([MachineKey]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineLabel] ON [cso2].[DimMachine]([MachineLabel]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineName] ON [cso2].[DimMachine]([MachineName]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineOS] ON [cso2].[DimMachine]([MachineOS]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineSoftware] ON [cso2].[DimMachine]([MachineSoftware]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineSource] ON [cso2].[DimMachine]([MachineSource]);
CREATE STATISTICS [stat_cso2_DimMachine_MachineType] ON [cso2].[DimMachine]([MachineType]);
CREATE STATISTICS [stat_cso2_DimMachine_ServiceStartDate] ON [cso2].[DimMachine]([ServiceStartDate]);
CREATE STATISTICS [stat_cso2_DimMachine_Status] ON [cso2].[DimMachine]([Status]);
CREATE STATISTICS [stat_cso2_DimMachine_StoreKey] ON [cso2].[DimMachine]([StoreKey]);
CREATE STATISTICS [stat_cso2_DimMachine_UpdateDate] ON [cso2].[DimMachine]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimMachine_VendorName] ON [cso2].[DimMachine]([VendorName]);
CREATE STATISTICS [stat_cso2_DimOutage_ETLLoadID] ON [cso2].[DimOutage]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimOutage_LoadDate] ON [cso2].[DimOutage]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimOutage_OutageDescription] ON [cso2].[DimOutage]([OutageDescription]);
CREATE STATISTICS [stat_cso2_DimOutage_OutageKey] ON [cso2].[DimOutage]([OutageKey]);
CREATE STATISTICS [stat_cso2_DimOutage_OutageLabel] ON [cso2].[DimOutage]([OutageLabel]);
CREATE STATISTICS [stat_cso2_DimOutage_OutageName] ON [cso2].[DimOutage]([OutageName]);
CREATE STATISTICS [stat_cso2_DimOutage_OutageSubType] ON [cso2].[DimOutage]([OutageSubType]);
CREATE STATISTICS [stat_cso2_DimOutage_OutageSubTypeDescription] ON [cso2].[DimOutage]([OutageSubTypeDescription]);
CREATE STATISTICS [stat_cso2_DimOutage_OutageType] ON [cso2].[DimOutage]([OutageType]);
CREATE STATISTICS [stat_cso2_DimOutage_OutageTypeDescription] ON [cso2].[DimOutage]([OutageTypeDescription]);
CREATE STATISTICS [stat_cso2_DimOutage_UpdateDate] ON [cso2].[DimOutage]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimProductCategory_ETLLoadID] ON [cso2].[DimProductCategory]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimProductCategory_LoadDate] ON [cso2].[DimProductCategory]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimProductCategory_ProductCategoryDescription] ON [cso2].[DimProductCategory]([ProductCategoryDescription]);
CREATE STATISTICS [stat_cso2_DimProductCategory_ProductCategoryKey] ON [cso2].[DimProductCategory]([ProductCategoryKey]);
CREATE STATISTICS [stat_cso2_DimProductCategory_ProductCategoryLabel] ON [cso2].[DimProductCategory]([ProductCategoryLabel]);
CREATE STATISTICS [stat_cso2_DimProductCategory_ProductCategoryName] ON [cso2].[DimProductCategory]([ProductCategoryName]);
CREATE STATISTICS [stat_cso2_DimProductCategory_UpdateDate] ON [cso2].[DimProductCategory]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimScenario_ETLLoadID] ON [cso2].[DimScenario]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimScenario_LoadDate] ON [cso2].[DimScenario]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimScenario_ScenarioDescription] ON [cso2].[DimScenario]([ScenarioDescription]);
CREATE STATISTICS [stat_cso2_DimScenario_ScenarioKey] ON [cso2].[DimScenario]([ScenarioKey]);
CREATE STATISTICS [stat_cso2_DimScenario_ScenarioLabel] ON [cso2].[DimScenario]([ScenarioLabel]);
CREATE STATISTICS [stat_cso2_DimScenario_ScenarioName] ON [cso2].[DimScenario]([ScenarioName]);
CREATE STATISTICS [stat_cso2_DimScenario_UpdateDate] ON [cso2].[DimScenario]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimPromotion_DiscountPercent] ON [cso2].[DimPromotion]([DiscountPercent]);
CREATE STATISTICS [stat_cso2_DimPromotion_EndDate] ON [cso2].[DimPromotion]([EndDate]);
CREATE STATISTICS [stat_cso2_DimPromotion_ETLLoadID] ON [cso2].[DimPromotion]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimPromotion_LoadDate] ON [cso2].[DimPromotion]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimPromotion_MaxQuantity] ON [cso2].[DimPromotion]([MaxQuantity]);
CREATE STATISTICS [stat_cso2_DimPromotion_MinQuantity] ON [cso2].[DimPromotion]([MinQuantity]);
CREATE STATISTICS [stat_cso2_DimPromotion_PromotionCategory] ON [cso2].[DimPromotion]([PromotionCategory]);
CREATE STATISTICS [stat_cso2_DimPromotion_PromotionDescription] ON [cso2].[DimPromotion]([PromotionDescription]);
CREATE STATISTICS [stat_cso2_DimPromotion_PromotionKey] ON [cso2].[DimPromotion]([PromotionKey]);
CREATE STATISTICS [stat_cso2_DimPromotion_PromotionLabel] ON [cso2].[DimPromotion]([PromotionLabel]);
CREATE STATISTICS [stat_cso2_DimPromotion_PromotionName] ON [cso2].[DimPromotion]([PromotionName]);
CREATE STATISTICS [stat_cso2_DimPromotion_PromotionType] ON [cso2].[DimPromotion]([PromotionType]);
CREATE STATISTICS [stat_cso2_DimPromotion_StartDate] ON [cso2].[DimPromotion]([StartDate]);
CREATE STATISTICS [stat_cso2_DimPromotion_UpdateDate] ON [cso2].[DimPromotion]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_EndDate] ON [cso2].[DimSalesTerritory]([EndDate]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_ETLLoadID] ON [cso2].[DimSalesTerritory]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_GeographyKey] ON [cso2].[DimSalesTerritory]([GeographyKey]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_LoadDate] ON [cso2].[DimSalesTerritory]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_SalesTerritoryCountry] ON [cso2].[DimSalesTerritory]([SalesTerritoryCountry]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_SalesTerritoryGroup] ON [cso2].[DimSalesTerritory]([SalesTerritoryGroup]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_SalesTerritoryKey] ON [cso2].[DimSalesTerritory]([SalesTerritoryKey]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_SalesTerritoryLabel] ON [cso2].[DimSalesTerritory]([SalesTerritoryLabel]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_SalesTerritoryLevel] ON [cso2].[DimSalesTerritory]([SalesTerritoryLevel]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_SalesTerritoryManager] ON [cso2].[DimSalesTerritory]([SalesTerritoryManager]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_SalesTerritoryName] ON [cso2].[DimSalesTerritory]([SalesTerritoryName]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_SalesTerritoryRegion] ON [cso2].[DimSalesTerritory]([SalesTerritoryRegion]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_StartDate] ON [cso2].[DimSalesTerritory]([StartDate]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_Status] ON [cso2].[DimSalesTerritory]([Status]);
CREATE STATISTICS [stat_cso2_DimSalesTerritory_UpdateDate] ON [cso2].[DimSalesTerritory]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimProductSubcategory_ETLLoadID] ON [cso2].[DimProductSubcategory]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimProductSubcategory_LoadDate] ON [cso2].[DimProductSubcategory]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimProductSubcategory_ProductCategoryKey] ON [cso2].[DimProductSubcategory]([ProductCategoryKey]);
CREATE STATISTICS [stat_cso2_DimProductSubcategory_ProductSubcategoryDescription] ON [cso2].[DimProductSubcategory]([ProductSubcategoryDescription]);
CREATE STATISTICS [stat_cso2_DimProductSubcategory_ProductSubcategoryKey] ON [cso2].[DimProductSubcategory]([ProductSubcategoryKey]);
CREATE STATISTICS [stat_cso2_DimProductSubcategory_ProductSubcategoryLabel] ON [cso2].[DimProductSubcategory]([ProductSubcategoryLabel]);
CREATE STATISTICS [stat_cso2_DimProductSubcategory_ProductSubcategoryName] ON [cso2].[DimProductSubcategory]([ProductSubcategoryName]);
CREATE STATISTICS [stat_cso2_DimProductSubcategory_UpdateDate] ON [cso2].[DimProductSubcategory]([UpdateDate]);
CREATE STATISTICS [stat_cso2_FactITMachine_Datekey] ON [cso2].[FactITMachine]([Datekey]);
CREATE STATISTICS [stat_cso2_FactITMachine_ITMachinekey] ON [cso2].[FactITMachine]([ITMachinekey]);
CREATE STATISTICS [stat_cso2_FactITMachine_MachineKey] ON [cso2].[FactITMachine]([MachineKey]);
CREATE STATISTICS [stat_cso2_FactInventory_CurrencyKey] ON [cso2].[FactInventory]([CurrencyKey]);
CREATE STATISTICS [stat_cso2_FactInventory_DateKey] ON [cso2].[FactInventory]([DateKey]);
CREATE STATISTICS [stat_cso2_FactInventory_InventoryKey] ON [cso2].[FactInventory]([InventoryKey]);
CREATE STATISTICS [stat_cso2_FactInventory_ProductKey] ON [cso2].[FactInventory]([ProductKey]);
CREATE STATISTICS [stat_cso2_FactInventory_StoreKey] ON [cso2].[FactInventory]([StoreKey]);
CREATE STATISTICS [stat_cso2_FactStrategyPlan_AccountKey] ON [cso2].[FactStrategyPlan]([AccountKey]);
CREATE STATISTICS [stat_cso2_FactStrategyPlan_CurrencyKey] ON [cso2].[FactStrategyPlan]([CurrencyKey]);
CREATE STATISTICS [stat_cso2_FactStrategyPlan_Datekey] ON [cso2].[FactStrategyPlan]([Datekey]);
CREATE STATISTICS [stat_cso2_FactStrategyPlan_EntityKey] ON [cso2].[FactStrategyPlan]([EntityKey]);
CREATE STATISTICS [stat_cso2_FactStrategyPlan_ProductCategoryKey] ON [cso2].[FactStrategyPlan]([ProductCategoryKey]);
CREATE STATISTICS [stat_cso2_FactStrategyPlan_ScenarioKey] ON [cso2].[FactStrategyPlan]([ScenarioKey]);
CREATE STATISTICS [stat_cso2_FactStrategyPlan_StrategyPlanKey] ON [cso2].[FactStrategyPlan]([StrategyPlanKey]);
CREATE STATISTICS [stat_cso2_FactSalesQuota_ChannelKey] ON [cso2].[FactSalesQuota]([ChannelKey]);
CREATE STATISTICS [stat_cso2_FactSalesQuota_CurrencyKey] ON [cso2].[FactSalesQuota]([CurrencyKey]);
CREATE STATISTICS [stat_cso2_FactSalesQuota_DateKey] ON [cso2].[FactSalesQuota]([DateKey]);
CREATE STATISTICS [stat_cso2_FactSalesQuota_ProductKey] ON [cso2].[FactSalesQuota]([ProductKey]);
CREATE STATISTICS [stat_cso2_FactSalesQuota_SalesQuotaKey] ON [cso2].[FactSalesQuota]([SalesQuotaKey]);
CREATE STATISTICS [stat_cso2_FactSalesQuota_ScenarioKey] ON [cso2].[FactSalesQuota]([ScenarioKey]);
CREATE STATISTICS [stat_cso2_FactSalesQuota_StoreKey] ON [cso2].[FactSalesQuota]([StoreKey]);
CREATE STATISTICS [stat_cso2_FactSales_channelKey] ON [cso2].[FactSales]([channelKey]);
CREATE STATISTICS [stat_cso2_FactSales_CurrencyKey] ON [cso2].[FactSales]([CurrencyKey]);
CREATE STATISTICS [stat_cso2_FactSales_DateKey] ON [cso2].[FactSales]([DateKey]);
CREATE STATISTICS [stat_cso2_FactSales_ProductKey] ON [cso2].[FactSales]([ProductKey]);
CREATE STATISTICS [stat_cso2_FactSales_PromotionKey] ON [cso2].[FactSales]([PromotionKey]);
CREATE STATISTICS [stat_cso2_FactSales_SalesKey] ON [cso2].[FactSales]([SalesKey]);
CREATE STATISTICS [stat_cso2_FactSales_StoreKey] ON [cso2].[FactSales]([StoreKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_CurrencyKey] ON [cso2].[FactOnlineSales]([CurrencyKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_CustomerKey] ON [cso2].[FactOnlineSales]([CustomerKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_DateKey] ON [cso2].[FactOnlineSales]([DateKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_OnlineSalesKey] ON [cso2].[FactOnlineSales]([OnlineSalesKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_ProductKey] ON [cso2].[FactOnlineSales]([ProductKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_PromotionKey] ON [cso2].[FactOnlineSales]([PromotionKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_StoreKey] ON [cso2].[FactOnlineSales]([StoreKey]);
CREATE STATISTICS [stat_cso2_DimCustomer_AddressLine1] ON [cso2].[DimCustomer]([AddressLine1]);
CREATE STATISTICS [stat_cso2_DimCustomer_AddressLine2] ON [cso2].[DimCustomer]([AddressLine2]);
CREATE STATISTICS [stat_cso2_DimCustomer_BirthDate] ON [cso2].[DimCustomer]([BirthDate]);
CREATE STATISTICS [stat_cso2_DimCustomer_CompanyName] ON [cso2].[DimCustomer]([CompanyName]);
CREATE STATISTICS [stat_cso2_DimCustomer_CustomerKey] ON [cso2].[DimCustomer]([CustomerKey]);
CREATE STATISTICS [stat_cso2_DimCustomer_CustomerLabel] ON [cso2].[DimCustomer]([CustomerLabel]);
CREATE STATISTICS [stat_cso2_DimCustomer_CustomerType] ON [cso2].[DimCustomer]([CustomerType]);
CREATE STATISTICS [stat_cso2_DimCustomer_DateFirstPurchase] ON [cso2].[DimCustomer]([DateFirstPurchase]);
CREATE STATISTICS [stat_cso2_DimCustomer_Education] ON [cso2].[DimCustomer]([Education]);
CREATE STATISTICS [stat_cso2_DimCustomer_EmailAddress] ON [cso2].[DimCustomer]([EmailAddress]);
CREATE STATISTICS [stat_cso2_DimCustomer_ETLLoadID] ON [cso2].[DimCustomer]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimCustomer_FirstName] ON [cso2].[DimCustomer]([FirstName]);
CREATE STATISTICS [stat_cso2_DimCustomer_Gender] ON [cso2].[DimCustomer]([Gender]);
CREATE STATISTICS [stat_cso2_DimCustomer_GeographyKey] ON [cso2].[DimCustomer]([GeographyKey]);
CREATE STATISTICS [stat_cso2_DimCustomer_HouseOwnerFlag] ON [cso2].[DimCustomer]([HouseOwnerFlag]);
CREATE STATISTICS [stat_cso2_DimCustomer_LastName] ON [cso2].[DimCustomer]([LastName]);
CREATE STATISTICS [stat_cso2_DimCustomer_LoadDate] ON [cso2].[DimCustomer]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimCustomer_MaritalStatus] ON [cso2].[DimCustomer]([MaritalStatus]);
CREATE STATISTICS [stat_cso2_DimCustomer_MiddleName] ON [cso2].[DimCustomer]([MiddleName]);
CREATE STATISTICS [stat_cso2_DimCustomer_NameStyle] ON [cso2].[DimCustomer]([NameStyle]);
CREATE STATISTICS [stat_cso2_DimCustomer_NumberCarsOwned] ON [cso2].[DimCustomer]([NumberCarsOwned]);
CREATE STATISTICS [stat_cso2_DimCustomer_NumberChildrenAtHome] ON [cso2].[DimCustomer]([NumberChildrenAtHome]);
CREATE STATISTICS [stat_cso2_DimCustomer_Occupation] ON [cso2].[DimCustomer]([Occupation]);
CREATE STATISTICS [stat_cso2_DimCustomer_Phone] ON [cso2].[DimCustomer]([Phone]);
CREATE STATISTICS [stat_cso2_DimCustomer_Suffix] ON [cso2].[DimCustomer]([Suffix]);
CREATE STATISTICS [stat_cso2_DimCustomer_Title] ON [cso2].[DimCustomer]([Title]);
CREATE STATISTICS [stat_cso2_DimCustomer_TotalChildren] ON [cso2].[DimCustomer]([TotalChildren]);
CREATE STATISTICS [stat_cso2_DimCustomer_UpdateDate] ON [cso2].[DimCustomer]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimCustomer_YearlyIncome] ON [cso2].[DimCustomer]([YearlyIncome]);
CREATE STATISTICS [stat_cso2_DimEmployee_BaseRate] ON [cso2].[DimEmployee]([BaseRate]);
CREATE STATISTICS [stat_cso2_DimEmployee_BirthDate] ON [cso2].[DimEmployee]([BirthDate]);
CREATE STATISTICS [stat_cso2_DimEmployee_CurrentFlag] ON [cso2].[DimEmployee]([CurrentFlag]);
CREATE STATISTICS [stat_cso2_DimEmployee_DepartmentName] ON [cso2].[DimEmployee]([DepartmentName]);
CREATE STATISTICS [stat_cso2_DimEmployee_EmailAddress] ON [cso2].[DimEmployee]([EmailAddress]);
CREATE STATISTICS [stat_cso2_DimEmployee_EmergencyContactName] ON [cso2].[DimEmployee]([EmergencyContactName]);
CREATE STATISTICS [stat_cso2_DimEmployee_EmergencyContactPhone] ON [cso2].[DimEmployee]([EmergencyContactPhone]);
CREATE STATISTICS [stat_cso2_DimEmployee_EmployeeKey] ON [cso2].[DimEmployee]([EmployeeKey]);
CREATE STATISTICS [stat_cso2_DimEmployee_EndDate] ON [cso2].[DimEmployee]([EndDate]);
CREATE STATISTICS [stat_cso2_DimEmployee_ETLLoadID] ON [cso2].[DimEmployee]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimEmployee_FirstName] ON [cso2].[DimEmployee]([FirstName]);
CREATE STATISTICS [stat_cso2_DimEmployee_Gender] ON [cso2].[DimEmployee]([Gender]);
CREATE STATISTICS [stat_cso2_DimEmployee_HireDate] ON [cso2].[DimEmployee]([HireDate]);
CREATE STATISTICS [stat_cso2_DimEmployee_LastName] ON [cso2].[DimEmployee]([LastName]);
CREATE STATISTICS [stat_cso2_DimEmployee_LoadDate] ON [cso2].[DimEmployee]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimEmployee_MaritalStatus] ON [cso2].[DimEmployee]([MaritalStatus]);
CREATE STATISTICS [stat_cso2_DimEmployee_MiddleName] ON [cso2].[DimEmployee]([MiddleName]);
CREATE STATISTICS [stat_cso2_DimEmployee_ParentEmployeeKey] ON [cso2].[DimEmployee]([ParentEmployeeKey]);
CREATE STATISTICS [stat_cso2_DimEmployee_PayFrequency] ON [cso2].[DimEmployee]([PayFrequency]);
CREATE STATISTICS [stat_cso2_DimEmployee_Phone] ON [cso2].[DimEmployee]([Phone]);
CREATE STATISTICS [stat_cso2_DimEmployee_SalariedFlag] ON [cso2].[DimEmployee]([SalariedFlag]);
CREATE STATISTICS [stat_cso2_DimEmployee_SalesPersonFlag] ON [cso2].[DimEmployee]([SalesPersonFlag]);
CREATE STATISTICS [stat_cso2_DimEmployee_StartDate] ON [cso2].[DimEmployee]([StartDate]);
CREATE STATISTICS [stat_cso2_DimEmployee_Status] ON [cso2].[DimEmployee]([Status]);
CREATE STATISTICS [stat_cso2_DimEmployee_Title] ON [cso2].[DimEmployee]([Title]);
CREATE STATISTICS [stat_cso2_DimEmployee_UpdateDate] ON [cso2].[DimEmployee]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimEmployee_VacationHours] ON [cso2].[DimEmployee]([VacationHours]);
CREATE STATISTICS [stat_cso2_DimEntity_EndDate] ON [cso2].[DimEntity]([EndDate]);
CREATE STATISTICS [stat_cso2_DimEntity_EntityDescription] ON [cso2].[DimEntity]([EntityDescription]);
CREATE STATISTICS [stat_cso2_DimEntity_EntityKey] ON [cso2].[DimEntity]([EntityKey]);
CREATE STATISTICS [stat_cso2_DimEntity_EntityLabel] ON [cso2].[DimEntity]([EntityLabel]);
CREATE STATISTICS [stat_cso2_DimEntity_EntityName] ON [cso2].[DimEntity]([EntityName]);
CREATE STATISTICS [stat_cso2_DimEntity_EntityType] ON [cso2].[DimEntity]([EntityType]);
CREATE STATISTICS [stat_cso2_DimEntity_ETLLoadID] ON [cso2].[DimEntity]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimEntity_LoadDate] ON [cso2].[DimEntity]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimEntity_ParentEntityKey] ON [cso2].[DimEntity]([ParentEntityKey]);
CREATE STATISTICS [stat_cso2_DimEntity_ParentEntityLabel] ON [cso2].[DimEntity]([ParentEntityLabel]);
CREATE STATISTICS [stat_cso2_DimEntity_StartDate] ON [cso2].[DimEntity]([StartDate]);
CREATE STATISTICS [stat_cso2_DimEntity_Status] ON [cso2].[DimEntity]([Status]);
CREATE STATISTICS [stat_cso2_DimEntity_UpdateDate] ON [cso2].[DimEntity]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimProduct_AvailableForSaleDate] ON [cso2].[DimProduct]([AvailableForSaleDate]);
CREATE STATISTICS [stat_cso2_DimProduct_BrandName] ON [cso2].[DimProduct]([BrandName]);
CREATE STATISTICS [stat_cso2_DimProduct_ClassID] ON [cso2].[DimProduct]([ClassID]);
CREATE STATISTICS [stat_cso2_DimProduct_ClassName] ON [cso2].[DimProduct]([ClassName]);
CREATE STATISTICS [stat_cso2_DimProduct_ColorID] ON [cso2].[DimProduct]([ColorID]);
CREATE STATISTICS [stat_cso2_DimProduct_ColorName] ON [cso2].[DimProduct]([ColorName]);
CREATE STATISTICS [stat_cso2_DimProduct_ETLLoadID] ON [cso2].[DimProduct]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimProduct_ImageURL] ON [cso2].[DimProduct]([ImageURL]);
CREATE STATISTICS [stat_cso2_DimProduct_LoadDate] ON [cso2].[DimProduct]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimProduct_Manufacturer] ON [cso2].[DimProduct]([Manufacturer]);
CREATE STATISTICS [stat_cso2_DimProduct_ProductDescription] ON [cso2].[DimProduct]([ProductDescription]);
CREATE STATISTICS [stat_cso2_DimProduct_ProductKey] ON [cso2].[DimProduct]([ProductKey]);
CREATE STATISTICS [stat_cso2_DimProduct_ProductLabel] ON [cso2].[DimProduct]([ProductLabel]);
CREATE STATISTICS [stat_cso2_DimProduct_ProductName] ON [cso2].[DimProduct]([ProductName]);
CREATE STATISTICS [stat_cso2_DimProduct_ProductSubcategoryKey] ON [cso2].[DimProduct]([ProductSubcategoryKey]);
CREATE STATISTICS [stat_cso2_DimProduct_ProductURL] ON [cso2].[DimProduct]([ProductURL]);
CREATE STATISTICS [stat_cso2_DimProduct_Size] ON [cso2].[DimProduct]([Size]);
CREATE STATISTICS [stat_cso2_DimProduct_SizeRange] ON [cso2].[DimProduct]([SizeRange]);
CREATE STATISTICS [stat_cso2_DimProduct_SizeUnitMeasureID] ON [cso2].[DimProduct]([SizeUnitMeasureID]);
CREATE STATISTICS [stat_cso2_DimProduct_Status] ON [cso2].[DimProduct]([Status]);
CREATE STATISTICS [stat_cso2_DimProduct_StockTypeID] ON [cso2].[DimProduct]([StockTypeID]);
CREATE STATISTICS [stat_cso2_DimProduct_StockTypeName] ON [cso2].[DimProduct]([StockTypeName]);
CREATE STATISTICS [stat_cso2_DimProduct_StopSaleDate] ON [cso2].[DimProduct]([StopSaleDate]);
CREATE STATISTICS [stat_cso2_DimProduct_StyleID] ON [cso2].[DimProduct]([StyleID]);
CREATE STATISTICS [stat_cso2_DimProduct_StyleName] ON [cso2].[DimProduct]([StyleName]);
CREATE STATISTICS [stat_cso2_DimProduct_UnitCost] ON [cso2].[DimProduct]([UnitCost]);
CREATE STATISTICS [stat_cso2_DimProduct_UnitOfMeasureID] ON [cso2].[DimProduct]([UnitOfMeasureID]);
CREATE STATISTICS [stat_cso2_DimProduct_UnitOfMeasureName] ON [cso2].[DimProduct]([UnitOfMeasureName]);
CREATE STATISTICS [stat_cso2_DimProduct_UnitPrice] ON [cso2].[DimProduct]([UnitPrice]);
CREATE STATISTICS [stat_cso2_DimProduct_UpdateDate] ON [cso2].[DimProduct]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimProduct_Weight] ON [cso2].[DimProduct]([Weight]);
CREATE STATISTICS [stat_cso2_DimProduct_WeightUnitMeasureID] ON [cso2].[DimProduct]([WeightUnitMeasureID]);
CREATE STATISTICS [stat_cso2_DimAccount_AccountDescription] ON [cso2].[DimAccount]([AccountDescription]);
CREATE STATISTICS [stat_cso2_DimAccount_AccountKey] ON [cso2].[DimAccount]([AccountKey]);
CREATE STATISTICS [stat_cso2_DimAccount_AccountLabel] ON [cso2].[DimAccount]([AccountLabel]);
CREATE STATISTICS [stat_cso2_DimAccount_AccountName] ON [cso2].[DimAccount]([AccountName]);
CREATE STATISTICS [stat_cso2_DimAccount_AccountType] ON [cso2].[DimAccount]([AccountType]);
CREATE STATISTICS [stat_cso2_DimAccount_CustomMemberOptions] ON [cso2].[DimAccount]([CustomMemberOptions]);
CREATE STATISTICS [stat_cso2_DimAccount_CustomMembers] ON [cso2].[DimAccount]([CustomMembers]);
CREATE STATISTICS [stat_cso2_DimAccount_ETLLoadID] ON [cso2].[DimAccount]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimAccount_LoadDate] ON [cso2].[DimAccount]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimAccount_Operator] ON [cso2].[DimAccount]([Operator]);
CREATE STATISTICS [stat_cso2_DimAccount_ParentAccountKey] ON [cso2].[DimAccount]([ParentAccountKey]);
CREATE STATISTICS [stat_cso2_DimAccount_UpdateDate] ON [cso2].[DimAccount]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimAccount_ValueType] ON [cso2].[DimAccount]([ValueType]);
CREATE STATISTICS [stat_cso2_DimChannel_ChannelDescription] ON [cso2].[DimChannel]([ChannelDescription]);
CREATE STATISTICS [stat_cso2_DimChannel_ChannelKey] ON [cso2].[DimChannel]([ChannelKey]);
CREATE STATISTICS [stat_cso2_DimChannel_ChannelLabel] ON [cso2].[DimChannel]([ChannelLabel]);
CREATE STATISTICS [stat_cso2_DimChannel_ChannelName] ON [cso2].[DimChannel]([ChannelName]);
CREATE STATISTICS [stat_cso2_DimChannel_ETLLoadID] ON [cso2].[DimChannel]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimChannel_LoadDate] ON [cso2].[DimChannel]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimChannel_UpdateDate] ON [cso2].[DimChannel]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimCurrency_CurrencyDescription] ON [cso2].[DimCurrency]([CurrencyDescription]);
CREATE STATISTICS [stat_cso2_DimCurrency_CurrencyKey] ON [cso2].[DimCurrency]([CurrencyKey]);
CREATE STATISTICS [stat_cso2_DimCurrency_CurrencyLabel] ON [cso2].[DimCurrency]([CurrencyLabel]);
CREATE STATISTICS [stat_cso2_DimCurrency_CurrencyName] ON [cso2].[DimCurrency]([CurrencyName]);
CREATE STATISTICS [stat_cso2_DimCurrency_ETLLoadID] ON [cso2].[DimCurrency]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimCurrency_LoadDate] ON [cso2].[DimCurrency]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimCurrency_UpdateDate] ON [cso2].[DimCurrency]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimDate_AsiaSeason] ON [cso2].[DimDate]([AsiaSeason]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarDayOfWeek] ON [cso2].[DimDate]([CalendarDayOfWeek]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarDayOfWeekLabel] ON [cso2].[DimDate]([CalendarDayOfWeekLabel]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarHalfYear] ON [cso2].[DimDate]([CalendarHalfYear]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarHalfYearLabel] ON [cso2].[DimDate]([CalendarHalfYearLabel]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarMonth] ON [cso2].[DimDate]([CalendarMonth]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarMonthLabel] ON [cso2].[DimDate]([CalendarMonthLabel]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarQuarter] ON [cso2].[DimDate]([CalendarQuarter]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarQuarterLabel] ON [cso2].[DimDate]([CalendarQuarterLabel]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarWeek] ON [cso2].[DimDate]([CalendarWeek]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarWeekLabel] ON [cso2].[DimDate]([CalendarWeekLabel]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarYear] ON [cso2].[DimDate]([CalendarYear]);
CREATE STATISTICS [stat_cso2_DimDate_CalendarYearLabel] ON [cso2].[DimDate]([CalendarYearLabel]);
CREATE STATISTICS [stat_cso2_DimDate_DateDescription] ON [cso2].[DimDate]([DateDescription]);
CREATE STATISTICS [stat_cso2_DimDate_Datekey] ON [cso2].[DimDate]([Datekey]);
CREATE STATISTICS [stat_cso2_DimDate_EuropeSeason] ON [cso2].[DimDate]([EuropeSeason]);
CREATE STATISTICS [stat_cso2_DimDate_FiscalHalfYear] ON [cso2].[DimDate]([FiscalHalfYear]);
CREATE STATISTICS [stat_cso2_DimDate_FiscalHalfYearLabel] ON [cso2].[DimDate]([FiscalHalfYearLabel]);
CREATE STATISTICS [stat_cso2_DimDate_FiscalMonth] ON [cso2].[DimDate]([FiscalMonth]);
CREATE STATISTICS [stat_cso2_DimDate_FiscalMonthLabel] ON [cso2].[DimDate]([FiscalMonthLabel]);
CREATE STATISTICS [stat_cso2_DimDate_FiscalQuarter] ON [cso2].[DimDate]([FiscalQuarter]);
CREATE STATISTICS [stat_cso2_DimDate_FiscalQuarterLabel] ON [cso2].[DimDate]([FiscalQuarterLabel]);
CREATE STATISTICS [stat_cso2_DimDate_FiscalYear] ON [cso2].[DimDate]([FiscalYear]);
CREATE STATISTICS [stat_cso2_DimDate_FiscalYearLabel] ON [cso2].[DimDate]([FiscalYearLabel]);
CREATE STATISTICS [stat_cso2_DimDate_FullDateLabel] ON [cso2].[DimDate]([FullDateLabel]);
CREATE STATISTICS [stat_cso2_DimDate_HolidayName] ON [cso2].[DimDate]([HolidayName]);
CREATE STATISTICS [stat_cso2_DimDate_IsHoliday] ON [cso2].[DimDate]([IsHoliday]);
CREATE STATISTICS [stat_cso2_DimDate_IsWorkDay] ON [cso2].[DimDate]([IsWorkDay]);
CREATE STATISTICS [stat_cso2_DimDate_NorthAmericaSeason] ON [cso2].[DimDate]([NorthAmericaSeason]);
CREATE STATISTICS [stat_cso2_DimStore_AddressLine1] ON [cso2].[DimStore]([AddressLine1]);
CREATE STATISTICS [stat_cso2_DimStore_AddressLine2] ON [cso2].[DimStore]([AddressLine2]);
CREATE STATISTICS [stat_cso2_DimStore_CloseDate] ON [cso2].[DimStore]([CloseDate]);
CREATE STATISTICS [stat_cso2_DimStore_CloseReason] ON [cso2].[DimStore]([CloseReason]);
CREATE STATISTICS [stat_cso2_DimStore_EmployeeCount] ON [cso2].[DimStore]([EmployeeCount]);
CREATE STATISTICS [stat_cso2_DimStore_EntityKey] ON [cso2].[DimStore]([EntityKey]);
CREATE STATISTICS [stat_cso2_DimStore_ETLLoadID] ON [cso2].[DimStore]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimStore_GeographyKey] ON [cso2].[DimStore]([GeographyKey]);
CREATE STATISTICS [stat_cso2_DimStore_GeoLocation] ON [cso2].[DimStore]([GeoLocation]);
CREATE STATISTICS [stat_cso2_DimStore_Geometry] ON [cso2].[DimStore]([Geometry]);
CREATE STATISTICS [stat_cso2_DimStore_LastRemodelDate] ON [cso2].[DimStore]([LastRemodelDate]);
CREATE STATISTICS [stat_cso2_DimStore_LoadDate] ON [cso2].[DimStore]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimStore_OpenDate] ON [cso2].[DimStore]([OpenDate]);
CREATE STATISTICS [stat_cso2_DimStore_SellingAreaSize] ON [cso2].[DimStore]([SellingAreaSize]);
CREATE STATISTICS [stat_cso2_DimStore_Status] ON [cso2].[DimStore]([Status]);
CREATE STATISTICS [stat_cso2_DimStore_StoreDescription] ON [cso2].[DimStore]([StoreDescription]);
CREATE STATISTICS [stat_cso2_DimStore_StoreFax] ON [cso2].[DimStore]([StoreFax]);
CREATE STATISTICS [stat_cso2_DimStore_StoreKey] ON [cso2].[DimStore]([StoreKey]);
CREATE STATISTICS [stat_cso2_DimStore_StoreManager] ON [cso2].[DimStore]([StoreManager]);
CREATE STATISTICS [stat_cso2_DimStore_StoreName] ON [cso2].[DimStore]([StoreName]);
CREATE STATISTICS [stat_cso2_DimStore_StorePhone] ON [cso2].[DimStore]([StorePhone]);
CREATE STATISTICS [stat_cso2_DimStore_StoreType] ON [cso2].[DimStore]([StoreType]);
CREATE STATISTICS [stat_cso2_DimStore_UpdateDate] ON [cso2].[DimStore]([UpdateDate]);
CREATE STATISTICS [stat_cso2_DimStore_ZipCode] ON [cso2].[DimStore]([ZipCode]);
CREATE STATISTICS [stat_cso2_DimStore_ZipCodeExtension] ON [cso2].[DimStore]([ZipCodeExtension]);
CREATE STATISTICS [stat_cso2_DimGeography_CityName] ON [cso2].[DimGeography]([CityName]);
CREATE STATISTICS [stat_cso2_DimGeography_ContinentName] ON [cso2].[DimGeography]([ContinentName]);
CREATE STATISTICS [stat_cso2_DimGeography_ETLLoadID] ON [cso2].[DimGeography]([ETLLoadID]);
CREATE STATISTICS [stat_cso2_DimGeography_GeographyKey] ON [cso2].[DimGeography]([GeographyKey]);
CREATE STATISTICS [stat_cso2_DimGeography_GeographyType] ON [cso2].[DimGeography]([GeographyType]);
CREATE STATISTICS [stat_cso2_DimGeography_LoadDate] ON [cso2].[DimGeography]([LoadDate]);
CREATE STATISTICS [stat_cso2_DimGeography_RegionCountryName] ON [cso2].[DimGeography]([RegionCountryName]);
CREATE STATISTICS [stat_cso2_DimGeography_StateProvinceName] ON [cso2].[DimGeography]([StateProvinceName]);
CREATE STATISTICS [stat_cso2_DimGeography_UpdateDate] ON [cso2].[DimGeography]([UpdateDate]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_CurrencyKey] ON [cso2].[FactOnlineSales]([CurrencyKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_CustomerKey] ON [cso2].[FactOnlineSales]([CustomerKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_DateKey] ON [cso2].[FactOnlineSales]([DateKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_OnlineSalesKey] ON [cso2].[FactOnlineSales]([OnlineSalesKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_ProductKey] ON [cso2].[FactOnlineSales]([ProductKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_PromotionKey] ON [cso2].[FactOnlineSales]([PromotionKey]);
CREATE STATISTICS [stat_cso2_FactOnlineSales_StoreKey] ON [cso2].[FactOnlineSales]([StoreKey]);

## Query SQL DW

In [3]:
SELECT  SUM(f.[SalesAmount]) AS [sales_by_brand_amount]
,       p.[BrandName]
FROM    [cso1].[FactOnlineSales] AS f
JOIN    [cso1].[DimProduct]      AS p ON f.[ProductKey] = p.[ProductKey]
GROUP BY p.[BrandName]

In [4]:
SELECT  SUM(f.[SalesAmount]) AS [sales_by_brand_amount]
,       p.[BrandName]
FROM    [cso2].[FactOnlineSales] AS f
JOIN    [cso2].[DimProduct]      AS p ON f.[ProductKey] = p.[ProductKey]
GROUP BY p.[BrandName]

## Cleanup

In [5]:
DROP TABLE [cso1].DimAccount;
DROP TABLE [cso1].DimChannel;
DROP TABLE [cso1].DimCurrency;
DROP TABLE [cso1].DimCustomer;
DROP TABLE [cso1].DimDate;
DROP TABLE [cso1].DimEmployee;
DROP TABLE [cso1].DimEntity;
DROP TABLE [cso1].DimGeography;
DROP TABLE [cso1].DimMachine;
DROP TABLE [cso1].DimOutage;
DROP TABLE [cso1].DimProduct;
DROP TABLE [cso1].DimProductCategory;
DROP TABLE [cso1].DimProductSubcategory;
DROP TABLE [cso1].DimPromotion;
DROP TABLE [cso1].DimSalesTerritory;
DROP TABLE [cso1].DimScenario;
DROP TABLE [cso1].DimStore;
DROP TABLE [cso1].FactExchangeRate;
DROP TABLE [cso1].FactInventory;
DROP TABLE [cso1].FactITMachine;
DROP TABLE [cso1].FactITSLA;
DROP TABLE [cso1].FactOnlineSales;
DROP TABLE [cso1].FactSales;
DROP TABLE [cso1].FactSalesQuota;
DROP TABLE [cso1].FactStrategyPlan;

In [6]:
DROP TABLE [cso2].DimAccount;
DROP TABLE [cso2].DimChannel;
DROP TABLE [cso2].DimCurrency;
DROP TABLE [cso2].DimCustomer;
DROP TABLE [cso2].DimDate;
DROP TABLE [cso2].DimEmployee;
DROP TABLE [cso2].DimEntity;
DROP TABLE [cso2].DimGeography;
DROP TABLE [cso2].DimMachine;
DROP TABLE [cso2].DimOutage;
DROP TABLE [cso2].DimProduct;
DROP TABLE [cso2].DimProductCategory;
DROP TABLE [cso2].DimProductSubcategory;
DROP TABLE [cso2].DimPromotion;
DROP TABLE [cso2].DimSalesTerritory;
DROP TABLE [cso2].DimScenario;
DROP TABLE [cso2].DimStore;
DROP TABLE [cso2].FactExchangeRate;
DROP TABLE [cso2].FactInventory;
DROP TABLE [cso2].FactITMachine;
DROP TABLE [cso2].FactITSLA;
DROP TABLE [cso2].FactOnlineSales;
DROP TABLE [cso2].FactSales;
DROP TABLE [cso2].FactSalesQuota;
DROP TABLE [cso2].FactStrategyPlan;

In [7]:
DROP EXTERNAL TABLE [adls1].DimAccount;
DROP EXTERNAL TABLE [adls1].DimChannel;
DROP EXTERNAL TABLE [adls1].DimCurrency;
DROP EXTERNAL TABLE [adls1].DimCustomer;
DROP EXTERNAL TABLE [adls1].DimDate;
DROP EXTERNAL TABLE [adls1].DimEmployee;
DROP EXTERNAL TABLE [adls1].DimEntity;
DROP EXTERNAL TABLE [adls1].DimGeography;
DROP EXTERNAL TABLE [adls1].DimMachine;
DROP EXTERNAL TABLE [adls1].DimOutage;
DROP EXTERNAL TABLE [adls1].DimProduct;
DROP EXTERNAL TABLE [adls1].DimProductCategory;
DROP EXTERNAL TABLE [adls1].DimProductSubcategory;
DROP EXTERNAL TABLE [adls1].DimPromotion;
DROP EXTERNAL TABLE [adls1].DimSalesTerritory;
DROP EXTERNAL TABLE [adls1].DimScenario;
DROP EXTERNAL TABLE [adls1].DimStore;
DROP EXTERNAL TABLE [adls1].FactExchangeRate;
DROP EXTERNAL TABLE [adls1].FactInventory;
DROP EXTERNAL TABLE [adls1].FactITMachine;
DROP EXTERNAL TABLE [adls1].FactITSLA;
DROP EXTERNAL TABLE [adls1].FactOnlineSales;
DROP EXTERNAL TABLE [adls1].FactSales;
DROP EXTERNAL TABLE [adls1].FactSalesQuota;
DROP EXTERNAL TABLE [adls1].FactStrategyPlan;

In [8]:
DROP EXTERNAL TABLE [adls2].DimAccount;
DROP EXTERNAL TABLE [adls2].DimChannel;
DROP EXTERNAL TABLE [adls2].DimCurrency;
DROP EXTERNAL TABLE [adls2].DimCustomer;
DROP EXTERNAL TABLE [adls2].DimDate;
DROP EXTERNAL TABLE [adls2].DimEmployee;
DROP EXTERNAL TABLE [adls2].DimEntity;
DROP EXTERNAL TABLE [adls2].DimGeography;
DROP EXTERNAL TABLE [adls2].DimMachine;
DROP EXTERNAL TABLE [adls2].DimOutage;
DROP EXTERNAL TABLE [adls2].DimProduct;
DROP EXTERNAL TABLE [adls2].DimProductCategory;
DROP EXTERNAL TABLE [adls2].DimProductSubcategory;
DROP EXTERNAL TABLE [adls2].DimPromotion;
DROP EXTERNAL TABLE [adls2].DimSalesTerritory;
DROP EXTERNAL TABLE [adls2].DimScenario;
DROP EXTERNAL TABLE [adls2].DimStore;
DROP EXTERNAL TABLE [adls2].FactExchangeRate;
DROP EXTERNAL TABLE [adls2].FactInventory;
DROP EXTERNAL TABLE [adls2].FactITMachine;
DROP EXTERNAL TABLE [adls2].FactITSLA;
DROP EXTERNAL TABLE [adls2].FactOnlineSales;
DROP EXTERNAL TABLE [adls2].FactSales;
DROP EXTERNAL TABLE [adls2].FactSalesQuota;
DROP EXTERNAL TABLE [adls2].FactStrategyPlan;

In [9]:
DROP EXTERNAL FILE FORMAT TextFileFormat;

DROP EXTERNAL DATA SOURCE AzureDataLakeStorageGen1;
DROP EXTERNAL DATA SOURCE AzureDataLakeStorageGen2;

DROP DATABASE SCOPED CREDENTIAL AADCredential;
DROP DATABASE SCOPED CREDENTIAL ADLSStorageKey;